In [ ]:
# data visualization with plotly
import plotly.express as px
import pandas as pd

data = pd.read_csv('../data/data_by_sector_last_12_months.csv')

In [ ]:
# show columns in the data

for col in data.columns:
    print(col, type(data[col][0]))

In [ ]:
# competences
data['competences'].value_counts()

In [ ]:
# from data folder in the root directory i want to concatenate all the csv files
import os
import glob
import pandas as pd

path = '../data'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_from_each_file = (pd.read_csv(f) for f in all_files)

concatenated_df = pd.concat(df_from_each_file, ignore_index=True)


In [ ]:
# drop duplicates using the id column

concatenated_df.drop_duplicates(subset="id", keep="first", inplace=True)

concatenated_df

In [ ]:
concatenated_df["competences"].value_counts()

# Rome

In [ ]:
import requests

# Set your API key here
api_key = 'your_api_key_here'

# API endpoints
base_url = 'https://api.emploi-store.fr/partenaire/rome/v1/'
competences_url = f'{base_url}competences'
metiers_url = f'{base_url}metiers'
contextes_travail_url = f'{base_url}contextes_travail'
fiches_metiers_url = f'{base_url}fiches_metiers'

# Headers for authentication
headers = {'Authorization': f'Bearer {api_key}'}

# Fetch competences
response_competences = requests.get(competences_url, headers=headers)
competences_data = response_competences.json()

# Fetch metiers (jobs)
response_metiers = requests.get(metiers_url, headers=headers)
metiers_data = response_metiers.json()

# Fetch contextes de travail (work contexts)
response_contextes_travail = requests.get(contextes_travail_url, headers=headers)
contextes_travail_data = response_contextes_travail.json()

# Fetch fiches métiers (job sheets)
response_fiches_metiers = requests.get(fiches_metiers_url, headers=headers)
fiches_metiers_data = response_fiches_metiers.json()

# Now, you can process these data according to your project needs


In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import pandas as pd
# save as concatenated_offers.csv
concatenated_df = pd.read_csv('../data/concatenated_offers.csv')
# Sample data
df = concatenated_df

# convert series to a dataframe
df = pd.DataFrame(df, columns=df.columns)

import pandas as pd
import ast  # Import the abstract syntax tree module

# Function to convert string representation of lists of dictionaries back into actual lists of dictionaries
def convert_string_to_list(string):
    try:
        # Safely evaluate the string as a Python expression
        return ast.literal_eval(string)
    except (ValueError, SyntaxError):
        # In case of error (e.g., if the string is not a valid Python literal), return an empty list or the original string
        return []

# Assuming 'df' is your DataFrame and 'competences' is the column with the string representations
# Apply the conversion function to each element in the 'competences' column
df['competences'] = df['competences'].apply(convert_string_to_list)

# Now your 'competences' column should contain actual lists of dictionaries. You can apply your previous transformations.


C:\Users\Moham\AppData\Local\Temp\ipykernel_13240\458546508.py:6: DtypeWarning: Columns (39,61) have mixed types. Specify dtype option on import or set low_memory=False.
  concatenated_df = pd.read_csv('../data/concatenated_offers.csv')


In [5]:
def join_libelle(comp_list):
    # Extract 'libelle' from each dictionary in the list and join them with a space
    return " ".join([comp['libelle'] for comp in comp_list if 'libelle' in comp])

In [6]:
#df['competence_processed'] = df['competences'].apply(join_libelle)

df['competence_processed'] = df['competences'].apply(join_libelle)

In [7]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer


def teckeniser(texte): 
    words = word_tokenize(texte)

    stemmer_en = SnowballStemmer("english")
    stemmer_fr = SnowballStemmer("french")

    stopwords_fr_en= stopwords.words("english") + stopwords.words("french")
    # Stemming pour les mots anglais
    stemmed_anglais = set([stemmer_en.stem(word.lower()) for word in words if word.lower() not in stopwords_fr_en and word not in string.punctuation])
    # Stemming pour les mots français
    stemmed_francais = set([stemmer_fr.stem(word.lower()) for word in words if word.lower() not in stopwords_fr_en and word not in string.punctuation])

    return stemmed_anglais & stemmed_francais

In [8]:
df['competence_processed'] = df['competence_processed'].apply(teckeniser)

In [9]:
df['competence_processed'][0]

{'ajust',
 'analys',
 'besoin',
 'comprendr',
 'compt',
 'conduir',
 'consign',
 "d'un",
 'identifi',
 "l'état",
 'laboratoir',
 'mesur',
 'notion',
 'produit',
 'protocol',
 'rapport',
 'recherch',
 'registr',
 'relev',
 'rendu',
 'rendus',
 'réalis',
 'stock',
 'techniqu',
 'test',
 'échantillon'}

In [10]:
import re
import pdfplumber

def lirePDF(chemin_acces):
    # Extraire le texte du PDF
    with pdfplumber.open(chemin_acces) as pdf:
        texte = ""
        for page in pdf.pages:
            texte += page.extract_text()

    texte= re.sub(r'http\S+', ' ', texte) # enlever liens(texte)

    texte= re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', ' ', texte) # enlever adresses email

    texte= re.sub(r'[0-9()+-]', ' ', texte) #enlever numeros et symboles

    texte= re.sub(r'[^\w\s]', ' ', texte) # enlever symboles
    
    return texte

In [11]:
path ="C:\\Users\\Moham\\Desktop\\cv_fr.pdf"

texte = """Un étudiant en ingénierie à la recherche d'un emploi en aéronautique est généralement passionné, motivé et doté d'une solide formation technique. Il possède des compétences en conception assistée par ordinateur (CAO), en dynamique des fluides numérique (CFD) et en matériaux composites, essentielles dans la construction et la maintenance des aéronefs. Sa compréhension des principes de l'aérodynamique, des systèmes de propulsion et de l'électronique embarquée le rend apte à aborder des défis complexes. Il est souvent familiarisé avec des logiciels spécifiques comme CATIA ou SolidWorks et des langages de programmation comme Python ou C++. La gestion de projet, la résolution de problèmes et les compétences en communication sont également cruciales pour collaborer efficacement au sein d'équipes multidisciplinaires. Enfin, il est constamment à la recherche de nouvelles technologies et innovations dans le secteur aéronautique pour améliorer la sécurité et l'efficacité des appareils."""

In [12]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib

# Your DataFrame and preprocessing steps should go here
# df['competence_processed'] ...
# Ensure all entries in 'competence_processed' are strings
df['competence_processed'] = df['competence_processed'].apply(lambda x: ' '.join(x) if isinstance(x, set) else x)

# Initialize and fit the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['competence_processed'])

# Save the vectorized matrix and the vectorizer for later use
joblib.dump(tfidf_matrix, 'tfidf_matrix.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')

# Function to calculate the vector for new input text using the saved TF-IDF vectorizer
def vectorize_text(input_text):
    # Load the vectorizer
    loaded_vectorizer = joblib.load('tfidf_vectorizer.pkl')
    # Transform the input text
    input_text_vectorized = loaded_vectorizer.transform([input_text])
    return input_text_vectorized

# Function to calculate similarities between the input text vector and the TF-IDF matrix
def calculate_similarities(input_text_vectorized):
    # Load the TF-IDF matrix
    loaded_tfidf_matrix = joblib.load('tfidf_matrix.pkl')
    # Compute cosine similarities between input text and dataset
    similarities = cosine_similarity(input_text_vectorized, loaded_tfidf_matrix)
    return similarities

# Example usage:
input_text = texte
input_vectorized = vectorize_text(input_text)
similarities = calculate_similarities(input_vectorized)

# You can then use the 'similarities' array to find the most similar items


In [13]:
# If 'similarities' is an array or a list of lists with only one element
similarities = similarities[0]  # Select the first (and only) row to make it 1D


In [14]:
# Convert the 1D similarities array to a DataFrame
df_similarities = pd.DataFrame(similarities, columns=['similarity'])

# Concatenate the original DataFrame with the similarities DataFrame
similarity_df= pd.concat([df, df_similarities], axis=1)

# Sort the DataFrame by the similarities
similarity_df = similarity_df.sort_values(by='similarity', ascending=False)


In [15]:
similarity_df.head(10)

,id,intitule,description,dateCreation,dateActualisation,romeCode,romeLibelle,appellationlibelle,typeContrat,typeContratLibelle,...,contact.urlPostulation,contact.telephone,origineOffre.partenaires,conditionExercice,agence.telephone,Secteur,Date,Unnamed: 0,competence_processed,similarity
27809,170RCWN,Opérateur/opératrice presses (H/F),"Dans le respect des standards de qualité, déla...",2024-03-07T08:30:10.000Z,2024-03-11T09:35:09.000Z,H2202,Conduite d'équipement de fabrication de l'ameu...,Opérateur / Opératrice sur presses à bois,CDD,Contrat à durée déterminée - 1 Mois,...,NaN,NaN,NaN,NaN,NaN,16.0,2024-02-10 to 2024-03-11,NaN,et/ou lign machin,0.385712
42627,169MVJR,Technicien / Technicienne d'intervention en fr...,Vous devrez accomplir les taches suivantes:\n\...,2024-02-15T15:09:55.000Z,2024-02-15T15:09:56.000Z,I1306,"Installation et maintenance en froid, conditio...",Technicien(ne) d'intervention en froid et clim...,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,NaN,43.0,2024-01-16 to 2024-02-15,4947.0,techniqu et/ou réalis lign machin,0.366003
14125,166WPPT,Opérateur de production (H/F),Votre agence Job&Box Cherbourg recrute des opé...,2023-12-28T10:41:32.000Z,2024-01-11T08:51:18.000Z,H3101,Conduite d'équipement de fabrication de papier...,Opérateur / Opératrice de production de papier...,MIS,Mission intérimaire - 1 Mois,...,http://app.mytalentplug.com/description-offre....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lign product et/ou identifi d'un machin,0.349382
39530,167YHCV,Opérateur / Opératrice de production de papier...,Nous recrutons différents postes d'opérateurs ...,2024-01-18T15:54:20.000Z,2024-02-13T05:21:28.000Z,H3101,Conduite d'équipement de fabrication de papier...,Opérateur / Opératrice de production de papier...,CDD,Contrat à durée déterminée - 6 Mois,...,NaN,NaN,NaN,NaN,NaN,17.0,2024-01-16 to 2024-02-15,1651.0,lign product et/ou identifi d'un machin,0.349382
51650,169MBXP,OPERATEUR DE PRODUCTION (H/F),ADWORK'S TRAVAIL TEMPORAIRE recherche UN OPERA...,2024-02-15T09:58:24.000Z,2024-02-15T09:58:27.000Z,H3101,Conduite d'équipement de fabrication de papier...,Opérateur / Opératrice de production de papier...,MIS,Mission intérimaire - 4 Mois,...,NaN,NaN,NaN,NaN,NaN,78.0,2024-01-16 to 2024-02-15,8434.0,d'un product et/ou identifi lign machin,0.349382
51702,169MBHM,Opérateur de production (H/F),Notre agence recherche pour l'un de ses client...,2024-02-15T09:48:20.000Z,2024-02-15T09:57:07.000Z,H3101,Conduite d'équipement de fabrication de papier...,Opérateur / Opératrice de production de papier...,MIS,Mission intérimaire - 6 Mois,...,NaN,NaN,NaN,NaN,NaN,78.0,2024-01-16 to 2024-02-15,8486.0,d'un product et/ou identifi lign machin,0.349382
31322,170VZPQ,Poseur / Poseuse de menuiseries extérieures,Nous sommes une entreprise de vente et pose de...,2024-03-11T11:33:44.000Z,2024-03-11T12:00:17.000Z,F1616,Poseur / Poseuse de menuiseries extérieures,Poseur / Poseuse de menuiseries extérieures,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,NaN,43.0,2024-02-10 to 2024-03-11,NaN,élément l'état et/ou réalis support fermetur,0.328559
28561,170TGFX,Poseur en menuiseries H/F / TAF,Employeur présent au salon TAF des Hautes Pyré...,2024-03-08T11:38:41.000Z,2024-03-08T11:38:56.000Z,F1616,Poseur / Poseuse de menuiseries extérieures,Poseur / Poseuse de menuiseries extérieures,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,NaN,22.0,2024-02-10 to 2024-03-11,NaN,élément l'état et/ou réalis support fermetur,0.328559
39435,169KXKZ,AIDE CONDUCTEUR/CONDUCTRICE MACHINE (H/F),Vous souhaitez vous épanouir dans une entrepri...,2024-02-14T11:41:04.000Z,2024-02-14T21:51:50.000Z,H3101,Conduite d'équipement de fabrication de papier...,Aide conducteur / conductrice de machines à pa...,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,NaN,17.0,2024-01-16 to 2024-02-15,1520.0,lign l'exploit gestion product pilot et/ou ide...,0.315810
28004,169TXCH,Conducteur / Conductrice de machines à papier ...,Vous conduisez les machines de découpe à plat ..

In [16]:
import gensim
from gensim.models import Word2Vec
import numpy as np
import gensim.downloader as api


def calculate_average_word2vec_optimized(token_lists, model, num_features = 300):
    feature_vecs = np.zeros((len(token_lists), num_features))
    for idx, tokens in enumerate(token_lists):
        model_vocabs = [model[token] for token in tokens if token in model.wv]
        if model_vocabs:
            feature_vecs[idx] = np.mean(model_vocabs, axis=0)
    return feature_vecs

# Example function to compare two sets of strings using Word2Vec vectors
def compare_word2vec_sets(set1, set2, model_path):
    # Load pre-trained Word2Vec model
    model = Word2Vec.load(model_path)

    # Calculate Word2Vec vectors for each set
    set1_vectors = calculate_average_word2vec_optimized(set1, model, model.vector_size)
    set2_vectors = calculate_average_word2vec_optimized(set2, model, model.vector_size)

    # Calculate cosine similarity between the two sets of vectors
    similarities = []
    for vec1 in set1_vectors:
        for vec2 in set2_vectors:
            similarity = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
            similarities.append(similarity)

    # Return average similarity score
    return np.mean(similarities)

In [17]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [18]:
from gensim.models.fasttext import FastText as FT_gensim
# Assuming the path to the downloaded 'cc.fr.300.bin' file is 'path/to/cc.fr.300.bin'
model = FT_gensim.load_fasttext_format('C:\\Users\\Moham\\Documents\\data_pole_emploi\\Data_Pole_Emploi\\cc.fr.300.bin.gz')


C:\Users\Moham\AppData\Local\Temp\ipykernel_13240\4039335337.py:3: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  model = FT_gensim.load_fasttext_format('C:\\Users\\Moham\\Documents\\data_pole_emploi\\Data_Pole_Emploi\\cc.fr.300.bin.gz')


In [19]:
tokenlist = df['competence_processed'] 
# Calculate the average Word2Vec vectors for the token lists

import numpy as np

def calculate_average_word2vec_optimized(token_lists, model):
    num_features = model.vector_size  # Typically 300 for FastText models
    feature_vecs = np.zeros((len(token_lists), num_features), dtype="float32")
    for idx, tokens in enumerate(token_lists):
        valid_tokens = [token for token in tokens if token in model.wv.key_to_index]
        if valid_tokens:
            model_vocabs = np.array([model.wv[token] for token in valid_tokens])
            feature_vecs[idx] = np.mean(model_vocabs, axis=0)
    return feature_vecs


word2vec_vectors = calculate_average_word2vec_optimized(tokenlist, model)

# Calculate the similarity between the input text and the token lists   

input_text = texte  # Example input text

# tokenize the input text using the word2vec model 
input_text_tokens = teckeniser(input_text)

# Calculate the average Word2Vec vectors for the input text
input_text_vector = calculate_average_word2vec_optimized([input_text_tokens], model)   

# Calculate the similarity between the input text and the token lists
similarities = []

for vec1 in word2vec_vectors:
    similarity = np.dot(vec1, input_text_vector[0]) / (np.linalg.norm(vec1) * np.linalg.norm(input_text_vector[0]))
    similarities.append(similarity) 
    
print(len(similarities))

# Add the similarities to the DataFrame
df['word2vec_similarity'] = similarities

# Sort the DataFrame by the similarities
df = df.sort_values(by='word2vec_similarity', ascending=False)  

df.head(10)


C:\Users\Moham\AppData\Local\Temp\ipykernel_13240\2101056148.py:33: RuntimeWarning: invalid value encountered in scalar divide
  similarity = np.dot(vec1, input_text_vector[0]) / (np.linalg.norm(vec1) * np.linalg.norm(input_text_vector[0]))


53318


,id,intitule,description,dateCreation,dateActualisation,romeCode,romeLibelle,appellationlibelle,typeContrat,typeContratLibelle,...,contact.urlPostulation,contact.telephone,origineOffre.partenaires,conditionExercice,agence.telephone,Secteur,Date,Unnamed: 0,competence_processed,word2vec_similarity
24283,163PSJJ,Assistant(e) juridique (H/F),Un cabinet d'avocats situé à MULHOUSE recrute ...,2023-10-20T09:11:24.000Z,2024-01-04T12:51:25.000Z,K1902,Collaboration juridique,Assistant / Assistante de cabinet juridique,CDD,Contrat à durée déterminée - 18 Mois,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,procédur,0.369847
27675,170LBYV,FORUM RETAIL DE LUXE: CONSEILLER DE VENTE BILI...,ORGANISATION FORUM RETAIL DE LUXE ORGANISE LE ...,2024-03-04T08:34:08.000Z,2024-03-10T23:27:17.000Z,D1214,Vente en habillement et accessoires de la pers...,Vendeur / Vendeuse en prêt-à-porter de luxe,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,NaN,15.0,2024-02-10 to 2024-03-11,NaN,procédur,0.369847
45423,169LXYY,Assistant / Assistante de service juridique,Profil recherché :\n- Titulaire d'un Master II...,2024-02-15T09:10:56.000Z,2024-02-15T15:52:21.000Z,K1902,Collaboration juridique,Assistant / Assistante de service juridique,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,NaN,69.0,2024-01-16 to 2024-02-15,7909.0,procédur,0.369847
40103,169LJDX,Customer Quality Window (H/F),Le groupe AGC est un fournisseur mondial de ve...,2024-02-14T15:10:09.000Z,2024-02-14T15:41:12.000Z,H1502,Management et ingénierie qualité industrielle,Assureur / Assureuse qualité industrie,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,NaN,23.0,2024-01-16 to 2024-02-15,2303.0,procédur,0.369847
50166,169LPXZ,VENDEUR RAYON JARDIN - MAGASIN WELDOM (H/F),POINT P recrute un/une VENDEUR RAYON JARDIN - ...,2024-02-14T16:55:29.000Z,2024-02-14T16:55:30.000Z,D1209,Vente de végétaux,Vendeur / Vendeuse en jardinerie,CDI,Contrat à durée indéterminée,...,https://joinus.saint-gobain.com/description?nP...,NaN,NaN,NaN,NaN,46.0,2024-01-16 to 2024-02-15,4604.0,procédur,0.369847
50184,169LNJJ,Vendeur / Vendeuse conseil en jardinerie,"Groupe Altitude, groupe coopératif agricole et...",2024-02-14T16:23:23.000Z,2024-02-15T10:04:30.000Z,D1209,Vente de végétaux,Vendeur / Vendeuse conseil en jardinerie,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,NaN,46.0,2024-01-16 to 2024-02-15,4622.0,procédur,0.369847
31739,170TTPM,Coordinateur (trice) QEDD (H/F),"Intégrez l'équipe QHE d'Elodie, et pilotez la ...",2024-03-08T15:58:07.000Z,2024-03-08T15:59:55.000Z,H1502,Management et ingénierie qualité industrielle,Responsable contrôle qualité en industrie,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,NaN,46.0,2024-02-10 to 2024-03-11,NaN,procédur,0.369847
16727,166CFLK,Équipier polyvalent / Équipière polyvalente de...,Nous recherchons pour notre Pizzeria un(e) emp...,2023-12-11T10:17:05.000Z,2024-01-09T09:33:51.000Z,G1603,Personnel polyvalent en restauration,Équipier(ère) polyvalent(e) de restauration ra...,CDD,Contrat à durée déterminée - 6 Mois,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,procédur,0.369847
45358,169LHMM,Gestionnaire recouvrement amiable (H/F) TAF TO...,Nous recherchons pour notre Direction Financiè...,2024-02-14T14:57:50.000Z,2024-02-15T10:39:28.000Z,D1403,Relation commerciale auprès de particuliers,Chargé / Chargée de recouvrement de créances,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,NaN,68.0,2024-01-16 to 2024-02-15,7844.0,procédur recouvr créanc,0.369501
15255,166QWRL,Chargé.e de recouvrement BtoB H/F (H/F),Nous recherchons un(e) Chargé(e) de recouvreme...,2023-12-21T10:35:58.000Z,2024-01-10T12:17:49.000Z,D1403,Relation commerciale auprès de particuliers,Chargé / Chargée de recouvrement de créances,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,procédur recouvr créanc,0.369501


In [20]:
import numpy as np

def calculate_average_textkernel_optimized(token_lists, textkernel_model):
    """
    Converts lists of tokens into their vector representations using a TextKernel model
    and computes the average vector for each list.

    Parameters:
    - token_lists: A list of lists, where each inner list contains tokens (words or phrases).
    - textkernel_model: The TextKernel model or mechanism used to convert tokens into vectors.

    Returns:
    - A list of numpy arrays, where each array is the average vector of the corresponding token list.
    """
    vectorized_lists = []
    
    for tokens in token_lists:
        # Initialize an empty matrix to store vectors for each token in the list
        token_vectors = []
        
        for token in tokens:
            # Convert each token to a vector using the TextKernel model
            # Note: You'll need to replace 'textkernel_model.vectorize(token)' with the actual method
            # provided by TextKernel for converting a token to a vector.
            vector = textkernel_model.vectorize(token)
            
            # Check if the vector is valid (not None, for example)
            if vector is not None:
                token_vectors.append(vector)
        
        # Compute the average vector if there are any valid vectors, else use a zero vector
        if token_vectors:
            average_vector = np.mean(token_vectors, axis=0)
        else:
            # Adapt the dimension 'N' to match the model's vector size
            average_vector = np.zeros(N)
        
        vectorized_lists.append(average_vector)
    
    return vectorized_lists


In [21]:
list_sectors = concatenated_df['secteurActiviteLibelle'].unique()

In [22]:
import spacy
from spacy.lang.fr import French
import pandas as pd

# Load the French language model for spaCy
nlp = spacy.load("fr_core_news_sm")

# Define your sectors (replace with actual sector names if needed)
sectors = ["Healthcare", "Construction", "IT", "Education", "Retail", "Transport"]  # Add all your sectors here

# Dictionary mapping named entities or keywords to sectors
# Update this based on your specific data and requirements
entity_to_sector = {
    "hôpital": "Healthcare",
    "école": "Education",
    "ordinateur": "IT",
    "construction": "Construction",
    "magasin": "Retail",
    "transport": "Transport"
}

# Function to tag text with sectors
def tag_text_with_sectors(text):
    # Process the text with spaCy
    doc = nlp(text)

    # Identify sectors based on entities in text
    identified_sectors = set()
    for entity in doc.ents:
        # Map entity to sector if possible
        sector = entity_to_sector.get(entity.text.lower())
        if sector:
            identified_sectors.add(sector)

    return list(identified_sectors)

# Sample text to tag
sample_text = "L'hôpital recherche de nouveaux infirmiers. Construction de nouvelles salles de classe."
tagged_sectors = tag_text_with_sectors(sample_text)

print("Tagged Sectors:", tagged_sectors)


Tagged Sectors: []


In [23]:
sectors_keywords = {
    "Technologie": ["ingénieur logiciel", "développeur web", "scientifique de données", "cybersécurité", "administrateur réseau", "développeur mobile", "architecte système", "UI/UX designer", "analyste système", "ingénieur cloud"],
    "Santé": ["infirmier", "médecin", "assistant médical", "pharmacien", "dentiste", "psychologue", "chirurgien", "technicien de laboratoire", "radiologue", "anesthésiste"],
    "Énergie": ["énergie renouvelable", "ingénieur solaire", "technicien éolien", "ingénieur pétrolier", "technicien de maintenance", "ingénieur nucléaire", "consultant en énergie", "analyste en énergie", "gestionnaire de projet énergie", "technicien HVAC"],
    "Finance": ["comptable", "analyste financier", "banquier d'investissement", "contrôleur de gestion", "auditeur", "gestionnaire de fonds", "conseiller financier", "courtier", "analyste de crédit", "gestionnaire de risques"],
    "Secteurs Émergents": ["IA", "blockchain", "robotique", "biotechnologie", "véhicules autonomes", "énergie géothermique", "impression 3D", "nanotechnologie"],
    "Enseignement": ["professeur", "conseiller pédagogique", "assistant d'éducation", "chercheur", "tuteur", "éducateur spécialisé", "formateur pour adultes", "professeur de langue", "bibliothécaire"],
    "Construction": ["chef de chantier", "architecte", "ingénieur civil", "ouvrier", "technicien en bâtiment", "maçon", "charpentier", "électricien", "plombier", "peintre"],
    "Juridique": ["avocat", "notaire", "juriste", "greffier", "conseiller juridique", "assistant juridique", "procureur", "juge", "clerc de notaire", "huissier de justice"],
    "Marketing": ["chef de produit", "responsable marketing", "analyste marché", "chargé de communication", "social media manager", "concepteur-rédacteur", "directeur artistique", "planneur stratégique", "responsable SEO", "data analyst marketing"],
    "Vente": ["vendeur", "représentant commercial", "responsable des ventes", "chef de secteur", "conseiller clientèle", "manager de rayon", "négociateur B2B", "délégué pharmaceutique", "directeur de magasin", "chargé d'affaires"],
    "Logistique": ["logisticien", "chauffeur poids lourd", "responsable d'entrepôt", "agent d'exploitation", "coordinateur logistique", "préparateur de commandes", "gestionnaire de stocks", "agent de transit", "affréteur"],
    "Agriculture": ["agronome", "exploitant agricole", "ouvrier agricole", "technicien agricole", "viticole", "paysagiste", "forestier", "aquaculteur", "conseiller en agriculture", "ingénieur agroalimentaire"],
    "Art et Culture": ["artiste", "conservateur de musée", "archéologue", "critique d'art", "directeur de galerie", "bibliothécaire", "historien de l'art", "régisseur d'œuvres"],
    "Tourisme": ["guide touristique", "agent de voyage", "réceptionniste", "chef de brigade", "animateur touristique", "gestionnaire de site touristique", "consultant en tourisme", "agent de réservation", "directeur d'hôtel", "organisateur d'événements"],
    "Restauration": ["chef de cuisine", "serveur", "barman", "pâtissier", "sommelier", "maître d'hôtel", "cuisinier", "boulanger", "commis de cuisine", "gérant de restaurant"],
    "Ingénierie": ["ingénieur mécanique", "ingénieur électronique", "ingénieur aéronautique", "ingénieur génie civil", "ingénieur en matériaux", "ingénieur électrique", "ingénieur biomédical", "ingénieur en robotique", "ingénieur environnemental", "ingénieur chimiste"],
    "Informatique et Télécommunications": ["technicien support", "administrateur système", "ingénieur télécom", "développeur de logiciels", "analyste de sécurité informatique", "ingénieur réseau", "architecte IT", "spécialiste en base de données", "consultant en systèmes d'information", "spécialiste en cloud computing"]
}



# Function to tag text with sectors based on keywords
def tag_sectors(text, sectors_keywords):
    # Convert all text to lower case for case insensitive matching
    text = text.lower()
    
    # Initialize a list to store matched sectors
    tagged_sectors = []
    
    # Loop through each sector and its keywords
    for sector, keywords in sectors_keywords.items():
        # Check if any of the sector's keywords are in the text
        if any(keyword.lower() in text for keyword in keywords):
            # If a keyword is found, add the sector to the tagged sectors
            tagged_sectors.append(sector)
    
    # Return the list of tagged sectors
    return tagged_sectors

# Example usage
example_text = """Un étudiant en ingénierie à la recherche d'un emploi en aéronautique est généralement passionné, motivé et doté d'une solide formation technique. Il possède des compétences en conception assistée par ordinateur (CAO), en dynamique des fluides numérique (CFD) et en matériaux composites, essentielles dans la construction et la maintenance des aéronefs. Sa compréhension des principes de l'aérodynamique, des systèmes de propulsion et de l'électronique embarquée le rend apte à aborder des défis complexes. Il est souvent familiarisé avec des logiciels spécifiques comme CATIA ou SolidWorks et des langages de programmation comme Python ou C++. La gestion de projet, la résolution de problèmes et les compétences en communication sont également cruciales pour collaborer efficacement au sein d'équipes multidisciplinaires. Enfin, il est constamment à la recherche de nouvelles technologies et innovations dans le secteur aéronautique pour améliorer la sécurité et l'efficacité des appareils."""
#example_text = "We are looking for an experienced ingénieur logiciel and data analyst to join our team."
tagged = tag_sectors(example_text, sectors_keywords)
print("Tagged Sectors:", tagged)


Tagged Sectors: ['Secteurs Émergents']


In [24]:
# remove nan from list_sector
list_sectors.tolist()

['Analyses, essais et inspections techniques',
 'Hébergement médicalisé pour adultes handicapés et autre hébergement médicalisé',
 'Hébergement médicalisé pour personnes âgées',
 'Ambulances',
 'Promotion immobilière de logements',
 'Hébergement social pour adultes et familles en difficultés et autre hébergement social',
 'Activités hospitalières',
 "Commerce de détail d'articles médicaux et orthopédiques en magasin spécialisé",
 'Travaux de terrassement courants et travaux préparatoires',
 'Hôtels et hébergement similaire',
 'Aide à domicile',
 'Action sociale sans hébergement n.c.a.',
 'Commerce de détail de produits pharmaceutiques en magasin spécialisé',
 "Captage, traitement et distribution d'eau",
 'Conseil pour les affaires et autres conseils de gestion',
 'Supermarchés',
 "Fabrication d'autres produits alimentaires n.c.a.",
 'Activités comptables',
 'Coiffure',
 'Aide par le travail',
 'Fabrication de cacao, chocolat et de produits de confiserie',
 'Agences immobilières',
 'Act

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Sample documents
documents = list_sectors #, "The medical sector requires many nurses", "Renewable energy is growing fast"]
vectorizer = TfidfVectorizer()

# Fit and transform the documents
tfidf_matrix = vectorizer.fit_transform(documents)

# Get feature names to check against sector keywords
feature_names = vectorizer.get_feature_names_out()

# Example of tagging based on TF-IDF scores
def tag_sectors_tfidf(feature_names, sectors_keywords):
    tagged_sectors = set()
    for sector, keywords in sectors_keywords.items():
        for keyword in keywords:
            if keyword.lower() in feature_names:
                tagged_sectors.add(sector)
                break
    return list(tagged_sectors)

# Usage
tagged_tfidf = tag_sectors_tfidf(feature_names, sectors_keywords)
print("Tagged Sectors (TF-IDF):", tagged_tfidf)


ValueError: np.nan is an invalid document, expected byte or unicode string.

In [26]:
listeGPT_competences_nouveau = [['langage développement web HTML CSS JavaScript Python PHP Ruby Java C#',
  'framework développement web Django Flask Express Angular React Vue.js Laravel Ruby on Rails ASP.NET',
  'base de données SQL MySQL PostgreSQL MongoDB Firebase',
  "conception d'interfaces utilisateur",
  "intégration d'API RESTful",
  'optimisation des performances web',
  'sécurité des applications web',
  "tests unitaires et tests d'intégration",
  'gestion de versions Git',
  'résolution de problèmes techniques',
  'développement web',
  'design responsive',
  'SEO'],
 ['langage programmation Java Python C++ C# JavaScript Ruby Swift Kotlin',
  'frameworks logiciel Spring Hibernate .NET Django Ruby on Rails React Angular Vue.js',
  "développement d'applications desktop et mobile",
  'architecture logicielle',
  "tests unitaires et tests d'intégration",
  'conception orientée objet',
  'gestion de bases de données relationnelles et NoSQL',
  'optimisation des performances logicielles',
  'résolution de bugs et débogage',
  'gestion de projets logiciels',
  'architecture microservices',
  'Intégration continue',
  'Livraison continue'],
 ['langage programmation Python R SQL Scala Julia MATLAB',
  'outils de manipulation de données Pandas NumPy SciPy',
  'bibliothèques de Machine Learning et Deep Learning TensorFlow PyTorch Scikit-learn Keras',
  'visualisation de données Matplotlib Seaborn Plotly Tableau',
  'analyse statistique',
  'modélisation prédictive',
  'traitement du langage naturel',
  'apprentissage automatique non supervisé',
  'gestion de bases de données Big Data',
  'communication des résultats',
  'Big Data',
  'data engineering',
  'cloud data services'],
 ['sécurité informatique',
  'tests de pénétration',
  'gestion des identités et des accès',
  'sécurité des réseaux',
  'cryptographie',
  'analyse des vulnérabilités',
  "détection d'intrusions",
  'gestion des incidents de sécurité',
  'conformité réglementaire',
  'formation à la sécurité',
  'forensics informatique',
  'sécurité des endpoints',
  'sécurité mobile'],
 ["administration de systèmes d'exploitation Linux Windows Unix",
  'configuration de réseaux locaux et étendus',
  'gestion des services réseau (DNS, DHCP, LDAP)',
  'sécurité des réseaux et des pare-feu',
  'virtualisation et conteneurisation',
  'sauvegarde et restauration de données',
  'supervision et surveillance des réseaux',
  'dépannage réseau',
  'documentation des infrastructures',
  'collaboration avec les équipes techniques',
  'SDN',
  'Zero Trust network',
  'automatisation réseau'],
 ['langage développement mobile Java Kotlin Swift',
  'framework développement mobile Android Studio Xcode',
  "conception d'interfaces utilisateur mobiles",
  "intégration d'API et services web",
  "tests unitaires et tests d'intégration",
  'débogage et optimisation des performances',
  'gestion des versions et déploiements',
  'veille technologique en développement mobile',
  'collaboration avec des équipes de développement',
  'Flutter',
  'React Native',
  'sécurité des applications mobiles'],
 ['conception architecture systèmes informatiques',
  'analyse des besoins et des exigences',
  'sélection de technologies et plateformes',
  'mise en œuvre de solutions évolutives',
  'sécurité et intégrité des données',
  'gestion des performances et de la capacité',
  'interopérabilité et intégration de systèmes',
  'documentation des architectures',
  'gestion des risques et conformité aux normes',
  'collaboration avec des équipes multidisciplinaires',
  'cloud hybride',
  'multi-cloud',
  'conteneurs'],
 ['conception interfaces utilisateur intuitives',
  'analyse des besoins des utilisateurs',
  'création de wireframes et maquettes',
  "tests d'utilisabilité et rétroaction",
  'compétences en design graphique',
  'adaptabilité aux tendances en UX/UI',
  'communication des concepts de design',
  'respect des délais et gestion de projet',
  'collaboration avec des équipes de développement et marketing',
  'design thinking',
  'Figma',
  'accessibilité web'],
 ['analyse besoins métier et processus',
  'documentation des exigences fonctionnelles et non fonctionnelles',
  'modélisation des processus métier et flux de données',
  'évaluation des systèmes existants',
  'coordination entre équipes métier et techniques',
  "tests d'acceptation utilisateur",
  'support technique et résolution des problèmes',
  'formation des utilisateurs finaux',
  'reporting et suivi des indicateurs de performance',
  'adaptabilité aux changements et évolutions des besoins',
  'Agile',
  'Scrum',
  'DevOps'],
 ['conception, déploiement et gestion architectures cloud',
  'automatisation des opérations (Infrastructure as Code)',
  'sécurité et conformité des données',
  'optimisation des coûts et performances',
  'haute disponibilité et reprise après sinistre',
  "migration d'applications vers le cloud",
  'gestion des identités et des accès (IAM)',
  'intégration de services cloud (IaaS, PaaS, SaaS)',
  'monitoring et surveillance des services cloud',
  "collaboration avec des équipes de développement et d'exploitation",
  'serverless',
  'architecture orientée services',
  'cloud hybride et multi-cloud'],
 ['soins infirmiers',
  'diagnostic et évaluation des patients',
  'administration de traitements médicaux',
  'suivi et surveillance des patients',
  'communication avec les patients et leurs familles',
  "coordination des soins avec d'autres professionnels de la santé",
  'respect des protocoles médicaux et des normes de sécurité',
  'rédaction de rapports et tenue de dossiers médicaux',
  'formation continue et développement professionnel',
  'empathie et soutien psychologique'],
 ['diagnostic médical',
  'traitement des maladies et des blessures',
  'prescription de médicaments',
  'suivi et gestion des patients',
  'intervention chirurgicale',
  'communication avec les patients et les familles',
  "collaboration avec d'autres professionnels de la santé",
  'recherche médicale et développement de nouvelles thérapies',
  'formation continue et participation à des conférences',
  'éthique médicale et respect des normes professionnelles'],
 ['assistance aux médecins et infirmières',
  'préparation des patients pour les consultations',
  'prise de rendez-vous et gestion des dossiers médicaux',
  'mesure de signes vitaux et collecte de données médicales',
  'administration de traitements et de médicaments',
  'accueil et orientation des patients',
  'communication avec les patients et leurs familles',
  "collaboration avec l'équipe médicale",
  'respect de la confidentialité et des protocoles',
  'formation continue et développement professionnel'],
 ['délivrance de médicaments sur ordonnance',
  'conseils pharmaceutiques aux patients',
  'gestion des stocks et des commandes',
  'préparation de médicaments',
  'vérification des interactions médicamenteuses',
  'surveillance des effets secondaires et des réactions allergiques',
  'communication avec les médecins et autres professionnels de la santé',
  'promotion de la santé et prévention des maladies',
  'respect des normes de sécurité et de confidentialité',
  'formation continue sur les nouveaux médicaments et traitements'],
 ['examen dentaire et diagnostic',
  'soins dentaires préventifs et curatifs',
  'traitement des caries et des infections dentaires',
  'extraction des dents et chirurgie buccale',
  'prothèses dentaires et implants',
  'orthodontie et correction des malocclusions',
  "conseils d'hygiène bucco-dentaire",
  'gestion des douleurs et des urgences dentaires',
  'communication avec les patients de tous âges',
  'formation continue sur les nouvelles techniques et technologies dentaires'],
 ['évaluation et diagnostic psychologique',
  'thérapie individuelle et de groupe',
  'conseils en santé mentale et bien-être',
  'gestion du stress et des émotions',
  'traitement des troubles psychologiques et des addictions',
  'support et orientation professionnelle',
  'intervention en cas de crise ou de traumatisme',
  'communication empathique et non-jugeante',
  "collaboration avec d'autres professionnels de la santé",
  'confidentialité et respect de la vie privée des patients'],
 ['intervention chirurgicale',
  'diagnostic et planification des opérations',
  "utilisation d'équipements médicaux et chirurgicaux",
  'gestion des risques et des complications',
  'soins post-opératoires et suivi des patients',
  'communication avec les patients et leurs familles',
  "collaboration avec d'autres chirurgiens et spécialistes",
  'formation continue sur les techniques chirurgicales',
  "respect des normes de sécurité et d'hygiène",
  'éthique médicale et consentement éclairé des patients'],
 ["collecte et préparation d'échantillons",
  'analyse et tests de laboratoire',
  "utilisation d'équipements de laboratoire",
  'interprétation de résultats',
  'documentation et tenue de dossiers',
  'respect des protocoles et des normes de sécurité',
  'maintenance et étalonnage des instruments',
  "collaboration avec d'autres techniciens et chercheurs",
  'communication des résultats aux professionnels de la santé',
  'formation continue sur les nouvelles techniques et technologies'],
 ["interprétation d'images médicales",
  "utilisation de technologies d'imagerie (radiographie, IRM, scanner)",
  'diagnostic et identification des anomalies',
  'communication des résultats aux médecins',
  "collaboration avec d'autres radiologues et techniciens",
  'gestion des dossiers et des données patients',
  'respect des normes de sécurité et de confidentialité',
  'formation continue sur les avancées en radiologie',
  'éthique médicale et consentement éclairé'],
 ["administration d'anesthésie générale et locale",
  "surveillance des signes vitaux pendant l'anesthésie",
  'gestion de la douleur post-opératoire',
  'prévention et traitement des complications',
  'communication avec les patients et les chirurgiens',
  "collaboration avec d'autres professionnels de la santé",
  "prise de décisions rapides en situation d'urgence",
  'maintenance et étalonnage des équipements',
  'formation continue sur les nouvelles techniques anesthésiques',
  'respect des protocoles et des normes de sécurité'],
 ["conception et développement de systèmes d'énergie renouvelable",
  "installation et maintenance d'équipements solaires et éoliens",
  "étude d'impact environnemental et évaluation des ressources",
  "optimisation de l'efficacité énergétique",
  "gestion de projets d'énergie renouvelable",
  'utilisation de logiciels de modélisation et de simulation',
  'respect des normes de sécurité et de conformité',
  'analyse économique et financière des projets',
  'communication avec les parties prenantes',
  'veille technologique sur les avancées en énergies renouvelables'],
 ['conception et développement de systèmes photovoltaïques',
  'étude de faisabilité et dimensionnement des installations',
  'utilisation de logiciels de simulation solaire',
  'installation et maintenance de panneaux solaires',
  'suivi de performance et diagnostic des systèmes',
  'gestion de projets solaires',
  'collaboration avec des ingénieurs et des techniciens',
  'veille technologique sur les avancées en énergie solaire',
  'formation continue sur les nouvelles technologies',
  'respect des normes de sécurité et environnementales'],
 ["installation et maintenance d'éoliennes",
  'diagnostic et réparation des équipements',
  "utilisation d'outils de mesure et de diagnostic",
  'gestion de projets éoliens',
  'travail en hauteur et en extérieur',
  'collaboration avec des ingénieurs et des équipes techniques',
  'respect des protocoles de sécurité',
  'formation continue sur les nouvelles technologies éoliennes',
  'documentation des interventions et des données',
  'communication avec les clients et les partenaires'],
 ['exploration et production de pétrole et de gaz',
  'analyse géologique et évaluation des réserves',
  "conception de puits de forage et d'équipements",
  'utilisation de techniques de forage et de fracturation',
  "gestion de projets d'extraction",
  'optimisation de la production et du rendement',
  "gestion des risques et de l'impact environnemental",
  'respect des normes de sécurité et de réglementation',
  'communication avec les équipes sur le terrain',
  'formation continue sur les nouvelles technologies et méthodes'],
 ['maintenance préventive et corrective des équipements',
  'diagnostic et dépannage des pannes',
  "utilisation d'outils et d'équipements de mesure",
  'gestion des stocks de pièces de rechange',
  'planification des interventions de maintenance',
  'documentation des activités de maintenance',
  'collaboration avec des équipes techniques',
  'respect des normes de sécurité et des procédures',
  'formation continue sur les nouvelles technologies',
  'communication avec les clients et les utilisateurs'],
 ['sécurité et sûreté nucléaire',
  'contrôle des réacteurs et des installations',
  'gestion des déchets radioactifs',
  'maintenance et réparation des équipements',
  'formation du personnel et supervision des opérations',
  "analyse des risques et planification d'urgence",
  'conformité aux réglementations et normes',
  'recherche et développement en énergie nucléaire',
  'communication avec les parties prenantes',
  'veille technologique sur les avancées nucléaires'],
 ['évaluation des besoins énergétiques',
  'audit et analyse de la consommation énergétique',
  "proposition de solutions d'efficacité énergétique",
  'mise en œuvre de mesures de réduction de la consommation',
  "gestion de projets d'optimisation énergétique",
  'utilisation de logiciels de simulation énergétique',
  'communication avec les clients et les partenaires',
  "formation et sensibilisation à l'économie d'énergie",
  'évaluation de la performance énergétique',
  'respect des réglementations et des normes environnementales'],
 ['collecte et analyse de données énergétiques',
  'modélisation et prévision de la consommation',
  'évaluation de la performance des installations',
  "identification des opportunités d'économie d'énergie",
  "rapports et recommandations pour l'efficacité énergétique",
  "utilisation de logiciels d'analyse et de simulation",
  'communication avec les clients et les décideurs',
  'suivi des tendances et des innovations énergétiques',
  'formation continue sur les méthodes et les technologies',
  'collaboration avec des équipes techniques et commerciales'],
 ['planification et organisation de projets énergétiques',
  'gestion des ressources et des budgets',
  'suivi des échéanciers et des objectifs',
  'coordination des équipes et des intervenants',
  'communication avec les clients et les parties prenantes',
  'gestion des risques et des imprévus',
  "rapports d'avancement et documentation des projets",
  'respect des réglementations et des normes',
  'évaluation de la performance et des résultats',
  'formation continue en gestion de projet et en énergie'],
 ['installation et maintenance de systèmes de chauffage',
  'ventilation et climatisation',
  'diagnostic et dépannage des systèmes HVAC',
  "utilisation d'outils de mesure et de test",
  'lecture de plans et de schémas techniques',
  'gestion des pièces de rechange et des stocks',
  'respect des normes de sécurité et de qualité',
  'formation des utilisateurs finaux',
  'collaboration avec des équipes techniques',
  'communication avec les clients et les fournisseurs'],
 ['tenue de livres et comptabilité générale',
  'analyse des états financiers',
  'déclarations fiscales et rapports réglementaires',
  'gestion des budgets et des prévisions',
  'audit comptable et contrôle interne',
  'gestion de trésorerie et recouvrement',
  'utilisation de logiciels de comptabilité',
  'communication avec les clients et les auditeurs',
  'formation continue sur les normes comptables',
  "respect de la confidentialité et de l'éthique professionnelle"],
 ['analyse des marchés financiers',
  'évaluation des investissements et des risques',
  'prévisions financières et modélisation',
  "rédaction de rapports d'analyse financière",
  "gestion de portefeuille et allocation d'actifs",
  "utilisation de logiciels d'analyse financière",
  'communication avec les investisseurs et les clients',
  'formation continue sur les marchés et les instruments financiers',
  'collaboration avec des équipes commerciales et de gestion',
  'respect des réglementations et des normes professionnelles'],
 ['structuration et négociation de transactions financières',
  'évaluation des entreprises et des projets',
  'montage financier et due diligence',
  'analyse des risques et des opportunités',
  'gestion de portefeuille et de risque',
  'communication avec les clients et les partenaires',
  "utilisation de modèles financiers et d'outils d'analyse",
  'formation continue sur les marchés financiers et les produits dérivés',
  'collaboration avec des équipes juridiques et commerciales',
  'respect des réglementations et des normes éthiques'],
 ['analyse des performances et des écarts',
  'prévisions budgétaires et planification stratégique',
  'reporting financier et tableau de bord',
  'optimisation des coûts et des investissements',
  'gestion des budgets et des prévisions',
  'évaluation des processus et des projets',
  'communication avec les départements opérationnels',
  'utilisation de logiciels de gestion et de business intelligence',
  'formation continue sur les techniques de contrôle de gestion',
  'collaboration avec des équipes financières et opérationnelles'],
 ['audit financier et opérationnel',
  'évaluation des risques et des contrôles internes',
  'vérification de la conformité aux normes et réglementations',
  'analyse des processus et des données',
  'révision des états financiers et des rapports',
  'communication des résultats et recommandations',
  "utilisation de logiciels d'audit et de gestion des risques",
  "formation continue sur les normes et les pratiques d'audit",
  "collaboration avec des équipes d'audit interne et externe",
  "respect de l'éthique et de la confidentialité professionnelle"],
 ["gestion de portefeuille et allocation d'actifs",
  "analyse des marchés financiers et des opportunités d'investissement",
  'sélection et suivi des placements',
  'relation clientèle et gestion de patrimoine',
  "communication des stratégies d'investissement",
  'utilisation de logiciels de gestion de fonds',
  'formation continue sur les tendances et les innovations en finance',
  "collaboration avec des équipes de recherche et d'analyse",
  'respect des réglementations et des normes éthiques',
  'performance et résultats financiers'],
 ['analyse de la situation financière des clients',
  'planification patrimoniale et fiscale',
  "recommandations d'investissement et de gestion de patrimoine",
  'élaboration de stratégies financières personnalisées',
  'suivi et ajustement des plans financiers',
  'communication des produits financiers et des risques',
  'relation clientèle et fidélisation',
  "utilisation d'outils de simulation financière",
  'formation continue sur les produits et les marchés financiers',
  'collaboration avec des équipes de conseillers et de vente'],
 ['recherche et analyse de produits financiers',
  "négociation et exécution d'ordres d'achat et de vente",
  "conseils sur les stratégies d'investissement",
  'gestion de portefeuille et de risque',
  'relation clientèle et prospection commerciale',
  'suivi des marchés financiers et des tendances',
  "utilisation de plateformes de trading et d'analyse",
  'formation continue sur les instruments financiers',
  'respect des réglementations et des normes professionnelles',
  'performance et résultats commerciaux'],
 ['évaluation de la solvabilité des emprunteurs',
  'analyse des états financiers et des données économiques',
  "notation de crédit et décision d'octroi",
  'gestion des risques et des garanties',
  'communication avec les clients et les partenaires bancaires',
  "utilisation de modèles de scoring et d'analyse",
  "formation continue sur les méthodes d'analyse de crédit",
  'collaboration avec des équipes commerciales et juridiques',
  'respect des politiques de crédit et des normes réglementaires',
  'performance et qualité des décisions de crédit'],
 ['identification et évaluation des risques financiers',
  'élaboration de politiques et de stratégies de gestion des risques',
  'mise en place de dispositifs de contrôle et de suivi',
  'analyse des scénarios et des impacts',
  'communication des recommandations aux décideurs',
  "utilisation d'outils d'analyse et de modélisation des risques",
  'formation continue sur les pratiques de gestion des risques',
  'collaboration avec des équipes de gestion et de conformité',
  'respect des réglementations et des normes internes',
  'performance et efficacité des dispositifs de gestion des risques'],
 ['apprentissage automatique (Machine Learning)',
  'réseaux neuronaux et deep learning',
  'traitement du langage naturel (NLP)',
  'vision par ordinateur',
  'reconnaissance de formes et de motifs',
  'algorithmes génétiques et optimisation',
  'systèmes experts et logique floue',
  'apprentissage non supervisé et clustering',
  "utilisation de bibliothèques d'IA (TensorFlow, PyTorch)",
  "développement d'applications et de solutions IA"],
 ['technologie de registre distribué (DLT)',
  'cryptomonnaies et tokens',
  'contrats intelligents (smart contracts)',
  'minage et validation de transactions',
  'consensus et preuve de travail/autorité/participation',
  'interopérabilité entre blockchains',
  'applications décentralisées (dApps)',
  'sécurité et confidentialité des données',
  'utilisation de frameworks blockchain (Ethereum, Hyperledger)',
  'développement de solutions blockchain'],
 ['conception et modélisation de robots',
  'programmation de robots et de systèmes embarqués',
  'capteurs et actionneurs',
  'navigation autonome et SLAM (Simultaneous Localization And Mapping)',
  'interaction homme-machine',
  'intelligence artificielle embarquée',
  "vision par ordinateur et reconnaissance d'objets",
  'systèmes de contrôle et de planification de mouvement',
  'applications industrielles et domestiques de la robotique',
  "collaboration avec des équipes d'ingénieurs et de chercheurs"],
 ['biologie moléculaire et cellulaire',
  'génétique et génomique',
  'culture de cellules et fermentation',
  "ingénierie génétique et manipulation de l'ADN",
  'bioprocédés et biofabrication',
  'analyse bioinformatique et modélisation',
  'diagnostic médical et biotechnologies médicales',
  'développement de médicaments et thérapies géniques',
  'biomatériaux et bio-impression 3D',
  'respect des normes de sécurité et des bonnes pratiques'],
 ['développement logiciel',
  'programmation',
  "conception d'architecture",
  'gestion de projet',
  'test et débogage',
  'maîtrise des langages de programmation (Java, Python, C++)',
  'connaissance des frameworks (Spring, Django, .NET)',
  'utilisation des outils de versioning (Git, SVN)',
  'résolution de problèmes complexes',
  'collaboration en équipe'],
 ['développement web',
  "conception d'interfaces utilisateur",
  'langages de programmation (HTML, CSS, JavaScript)',
  'frameworks web (React, Angular, Vue.js)',
  'bases de données (SQL, MongoDB)',
  'maîtrise des outils de développement (Visual Studio Code, Webpack)',
  'gestion de projets Agile',
  "sensibilité au design et à l'expérience utilisateur",
  'résolution de problèmes techniques',
  'veille technologique'],
 ['analyse de données',
  'exploration et nettoyage de données',
  'modélisation statistique et machine learning',
  'programmation en langage R et Python',
  'utilisation de bibliothèques (NumPy, pandas, scikit-learn)',
  'visualisation de données (Matplotlib, Seaborn, Plotly)',
  "compréhension des algorithmes d'apprentissage automatique",
  'traitement de données massives (Big Data)',
  'résolution de problèmes complexes',
  'communication des résultats'],
 ['sécurité informatique',
  'détection et prévention des intrusions',
  'analyse de vulnérabilités',
  'gestion des risques',
  'cryptographie',
  'sécurité des réseaux',
  "tests d'intrusion",
  'conformité réglementaire (RGPD, ISO 27001)',
  'sensibilisation à la sécurité',
  'gestion des incidents de sécurité'],
 ['administration système',
  'configuration et maintenance des réseaux',
  "sécurité des systèmes d'information",
  'supervision et surveillance des infrastructures',
  "résolution d'incidents réseau",
  'virtualisation (VMware, Hyper-V)',
  "gestion des sauvegardes et de la reprise d'activité",
  "gestion des utilisateurs et des droits d'accès",
  'veille technologique',
  'collaboration avec les équipes techniques'],
 ["développement d'applications mobiles",
  'langages de programmation (Java, Kotlin, Swift)',
  "conception d'interfaces utilisateur mobiles",
  'utilisation des SDK (Android, iOS)',
  "déploiement sur les stores d'applications",
  'test et débogage sur différentes plateformes',
  'optimisation des performances',
  'sécurité des applications mobiles',
  'collaboration avec les équipes de design et de backend',
  'adaptabilité aux évolutions des OS mobiles'],
 ['architecture informatique',
  'conception de solutions techniques',
  'intégration de systèmes',
  'scalabilité et haute disponibilité',
  'sécurité des systèmes',
  'virtualisation et cloud computing',
  'gestion des bases de données',
  'optimisation des performances',
  'veille technologique',
  'communication et présentation de solutions'],
 ["design d'interfaces utilisateur (UI)",
  'expérience utilisateur (UX)',
  'wireframing et prototypage',
  'conception graphique',
  'utilisabilité et accessibilité',
  'ergonomie des interfaces',
  'tests utilisateurs',
  'outils de design (Sketch, Adobe XD, Figma)',
  'collaboration avec les développeurs et les chefs de projet',
  'créativité et sensibilité artistique'],
 ['analyse des besoins',
  'étude de faisabilité',
  'spécifications fonctionnelles et techniques',
  'modélisation des processus métier',
  'gestion de projet',
  'tests et recettes',
  'formation des utilisateurs',
  'support technique',
  "résolution d'incidents",
  'documentation des systèmes'],
 ['cloud computing',
  'architecture cloud',
  'migration vers le cloud',
  'services cloud (AWS, Azure, Google Cloud)',
  'automatisation et orchestration',
  'sécurité cloud',
  'gestion des coûts',
  'performances et scalabilité',
  'administration des plateformes cloud',
  'collaboration en équipe']]

In [27]:
competencesGPT = [
    # Développeur Web
    [
        "langage développement web HTML CSS JavaScript Python PHP Ruby Java C#",
        "framework développement web Django Flask Express Angular React Vue.js Laravel Ruby on Rails ASP.NET",
        "base de données SQL MySQL PostgreSQL MongoDB Firebase",
        "conception d'interfaces utilisateur",
        "intégration d'API RESTful",
        "optimisation des performances web",
        "sécurité des applications web",
        "tests unitaires et tests d'intégration",
        "gestion de versions Git",
        "résolution de problèmes techniques"
    ],
    # Ingénieur Logiciel
    [
        "langage programmation Java Python C++ C# JavaScript Ruby Swift Kotlin",
        "frameworks logiciel Spring Hibernate .NET Django Ruby on Rails React Angular Vue.js",
        "développement d'applications desktop et mobile",
        "architecture logicielle",
        "tests unitaires et tests d'intégration",
        "conception orientée objet",
        "gestion de bases de données relationnelles et NoSQL",
        "optimisation des performances logicielles",
        "résolution de bugs et débogage",
        "gestion de projets logiciels"
    ],
    # Scientifique de Données
    [
        "langage programmation Python R SQL Scala Julia MATLAB",
        "outils de manipulation de données Pandas NumPy SciPy",
        "bibliothèques de Machine Learning et Deep Learning TensorFlow PyTorch Scikit-learn Keras",
        "visualisation de données Matplotlib Seaborn Plotly Tableau",
        "analyse statistique",
        "modélisation prédictive",
        "traitement du langage naturel",
        "apprentissage automatique non supervisé",
        "gestion de bases de données Big Data",
        "communication des résultats"
    ],
    # Cybersécurité
    [
        "sécurité informatique",
        "tests de pénétration",
        "gestion des identités et des accès",
        "sécurité des réseaux",
        "cryptographie",
        "analyse des vulnérabilités",
        "détection d'intrusions",
        "gestion des incidents de sécurité",
        "conformité réglementaire",
        "formation à la sécurité"
    ],
    # Administrateur Réseau
    [
        "administration de systèmes d'exploitation Linux Windows Unix",
        "configuration de réseaux locaux et étendus",
        "gestion des services réseau (DNS, DHCP, LDAP)",
        "sécurité des réseaux et des pare-feu",
        "virtualisation et conteneurisation",
        "sauvegarde et restauration de données",
        "supervision et surveillance des réseaux",
        "dépannage réseau",
        "documentation des infrastructures",
        "collaboration avec les équipes techniques"
    ],
    # Développeur Mobile
    [
        "langage développement mobile Java Kotlin Swift",
        "framework développement mobile Android Studio Xcode",
        "conception d'interfaces utilisateur mobiles",
        "intégration d'API et services web",
        "tests unitaires et tests d'intégration",
        "débogage et optimisation des performances",
        "gestion des versions et déploiements",
        "veille technologique en développement mobile",
        "collaboration avec des équipes de développement"
    ],
    # Architecte Système
    [
        "conception architecture systèmes informatiques",
        "analyse des besoins et des exigences",
        "sélection de technologies et plateformes",
        "mise en œuvre de solutions évolutives",
        "sécurité et intégrité des données",
        "gestion des performances et de la capacité",
        "interopérabilité et intégration de systèmes",
        "documentation des architectures",
        "gestion des risques et conformité aux normes",
        "collaboration avec des équipes multidisciplinaires"
    ],
    # UI/UX Designer
    [
        "conception interfaces utilisateur intuitives",
        "analyse des besoins des utilisateurs",
        "création de wireframes et maquettes",
        "tests d'utilisabilité et rétroaction",
        "compétences en design graphique",
        "adaptabilité aux tendances en UX/UI",
        "communication des concepts de design",
        "respect des délais et gestion de projet",
        "collaboration avec des équipes de développement et marketing"
    ],
    # Analyste Système
    [
        "analyse besoins métier et processus",
        "documentation des exigences fonctionnelles et non fonctionnelles",
        "modélisation des processus métier et flux de données",
        "évaluation des systèmes existants",
        "coordination entre équipes métier et techniques",
        "tests d'acceptation utilisateur",
        "support technique et résolution des problèmes",
        "formation des utilisateurs finaux",
        "reporting et suivi des indicateurs de performance",
        "adaptabilité aux changements et évolutions des besoins"
    ],
    # Ingénieur Cloud
    [
        "conception, déploiement et gestion architectures cloud",
        "automatisation des opérations (Infrastructure as Code)",
        "sécurité et conformité des données",
        "optimisation des coûts et performances",
        "haute disponibilité et reprise après sinistre",
        "migration d'applications vers le cloud",
        "gestion des identités et des accès (IAM)",
           "intégration de services cloud (IaaS, PaaS, SaaS)",
    "monitoring et surveillance des services cloud",
    "collaboration avec des équipes de développement et d'exploitation"
],
# Infirmier
    [
    "soins infirmiers",
    "diagnostic et évaluation des patients",
    "administration de traitements médicaux",
    "suivi et surveillance des patients",
    "communication avec les patients et leurs familles",
    "coordination des soins avec d'autres professionnels de la santé",
    "respect des protocoles médicaux et des normes de sécurité",
    "rédaction de rapports et tenue de dossiers médicaux",
    "formation continue et développement professionnel",
    "empathie et soutien psychologique"
    ],
# Médecin
    [
    "diagnostic médical",
    "traitement des maladies et des blessures",
    "prescription de médicaments",
    "suivi et gestion des patients",
    "intervention chirurgicale",
    "communication avec les patients et les familles",
    "collaboration avec d'autres professionnels de la santé",
    "recherche médicale et développement de nouvelles thérapies",
    "formation continue et participation à des conférences",
    "éthique médicale et respect des normes professionnelles"
    ],
# Assistant Médical
    [
    "assistance aux médecins et infirmières",
    "préparation des patients pour les consultations",
    "prise de rendez-vous et gestion des dossiers médicaux",
    "mesure de signes vitaux et collecte de données médicales",
    "administration de traitements et de médicaments",
    "accueil et orientation des patients",
    "communication avec les patients et leurs familles",
    "collaboration avec l'équipe médicale",
    "respect de la confidentialité et des protocoles",
    "formation continue et développement professionnel"
    ],
# Pharmacien
    [
    "délivrance de médicaments sur ordonnance",
    "conseils pharmaceutiques aux patients",
    "gestion des stocks et des commandes",
    "préparation de médicaments",
    "vérification des interactions médicamenteuses",
    "surveillance des effets secondaires et des réactions allergiques",
    "communication avec les médecins et autres professionnels de la santé",
    "promotion de la santé et prévention des maladies",
    "respect des normes de sécurité et de confidentialité",
    "formation continue sur les nouveaux médicaments et traitements"
    ],
# Dentiste
    [
    "examen dentaire et diagnostic",
    "soins dentaires préventifs et curatifs",
    "traitement des caries et des infections dentaires",
    "extraction des dents et chirurgie buccale",
    "prothèses dentaires et implants",
    "orthodontie et correction des malocclusions",
    "conseils d'hygiène bucco-dentaire",
    "gestion des douleurs et des urgences dentaires",
    "communication avec les patients de tous âges",
    "formation continue sur les nouvelles techniques et technologies dentaires"
    ],
# Psychologue
    [
    "évaluation et diagnostic psychologique",
    "thérapie individuelle et de groupe",
    "conseils en santé mentale et bien-être",
    "gestion du stress et des émotions",
    "traitement des troubles psychologiques et des addictions",
    "support et orientation professionnelle",
    "intervention en cas de crise ou de traumatisme",
    "communication empathique et non-jugeante",
    "collaboration avec d'autres professionnels de la santé",
    "confidentialité et respect de la vie privée des patients"
    ],
# Chirurgien
    [
    "intervention chirurgicale",
    "diagnostic et planification des opérations",
    "utilisation d'équipements médicaux et chirurgicaux",
    "gestion des risques et des complications",
    "soins post-opératoires et suivi des patients",
    "communication avec les patients et leurs familles",
    "collaboration avec d'autres chirurgiens et spécialistes",
    "formation continue sur les techniques chirurgicales",
    "respect des normes de sécurité et d'hygiène",
    "éthique médicale et consentement éclairé des patients"
    ],
# Technicien de Laboratoire
    [
    "collecte et préparation d'échantillons",
    "analyse et tests de laboratoire",
    "utilisation d'équipements de laboratoire",
    "interprétation de résultats",
    "documentation et tenue de dossiers",
    "respect des protocoles et des normes de sécurité",
    "maintenance et étalonnage des instruments",
    "collaboration avec d'autres techniciens et chercheurs",
    "communication des résultats aux professionnels de la santé",
    "formation continue sur les nouvelles techniques et technologies"],

#Radiologie
    [
    "interprétation d'images médicales",
    "utilisation de technologies d'imagerie (radiographie, IRM, scanner)",
    "diagnostic et identification des anomalies",
    "communication des résultats aux médecins",
    "collaboration avec d'autres radiologues et techniciens",
    "gestion des dossiers et des données patients",
    "respect des normes de sécurité et de confidentialité",
    "formation continue sur les avancées en radiologie",
    "éthique médicale et consentement éclairé"
    ],

    [
    "administration d'anesthésie générale et locale",
    "surveillance des signes vitaux pendant l'anesthésie",
    "gestion de la douleur post-opératoire",
    "prévention et traitement des complications",
    "communication avec les patients et les chirurgiens",
    "collaboration avec d'autres professionnels de la santé",
    "prise de décisions rapides en situation d'urgence",
    "maintenance et étalonnage des équipements",
    "formation continue sur les nouvelles techniques anesthésiques",
    "respect des protocoles et des normes de sécurité"
    ],

    [
    "conception et développement de systèmes d'énergie renouvelable",
    "installation et maintenance d'équipements solaires et éoliens",
    "étude d'impact environnemental et évaluation des ressources",
    "optimisation de l'efficacité énergétique",
    "gestion de projets d'énergie renouvelable",
    "utilisation de logiciels de modélisation et de simulation",
    "respect des normes de sécurité et de conformité",
    "analyse économique et financière des projets",
    "communication avec les parties prenantes",
    "veille technologique sur les avancées en énergies renouvelables"
    ],

    [
    "conception et développement de systèmes photovoltaïques",
    "étude de faisabilité et dimensionnement des installations",
    "utilisation de logiciels de simulation solaire",
    "installation et maintenance de panneaux solaires",
    "suivi de performance et diagnostic des systèmes",
    "gestion de projets solaires",
    "collaboration avec des ingénieurs et des techniciens",
    "veille technologique sur les avancées en énergie solaire",
    "formation continue sur les nouvelles technologies",
    "respect des normes de sécurité et environnementales"
    ],

    [
    "installation et maintenance d'éoliennes",
    "diagnostic et réparation des équipements",
    "utilisation d'outils de mesure et de diagnostic",
    "gestion de projets éoliens",
    "travail en hauteur et en extérieur",
    "collaboration avec des ingénieurs et des équipes techniques",
    "respect des protocoles de sécurité",
    "formation continue sur les nouvelles technologies éoliennes",
    "documentation des interventions et des données",
    "communication avec les clients et les partenaires"
    ],

    [
    "exploration et production de pétrole et de gaz",
    "analyse géologique et évaluation des réserves",
    "conception de puits de forage et d'équipements",
    "utilisation de techniques de forage et de fracturation",
    "gestion de projets d'extraction",
    "optimisation de la production et du rendement",
    "gestion des risques et de l'impact environnemental",
    "respect des normes de sécurité et de réglementation",
    "communication avec les équipes sur le terrain",
    "formation continue sur les nouvelles technologies et méthodes"
    ],

    [
    "maintenance préventive et corrective des équipements",
    "diagnostic et dépannage des pannes",
    "utilisation d'outils et d'équipements de mesure",
    "gestion des stocks de pièces de rechange",
    "planification des interventions de maintenance",
    "documentation des activités de maintenance",
    "collaboration avec des équipes techniques",
    "respect des normes de sécurité et des procédures",
    "formation continue sur les nouvelles technologies",
    "communication avec les clients et les utilisateurs"
    ],

    [
    "sécurité et sûreté nucléaire",
    "contrôle des réacteurs et des installations",
    "gestion des déchets radioactifs",
    "maintenance et réparation des équipements",
    "formation du personnel et supervision des opérations",
    "analyse des risques et planification d'urgence",
    "conformité aux réglementations et normes",
    "recherche et développement en énergie nucléaire",
    "communication avec les parties prenantes",
    "veille technologique sur les avancées nucléaires"
    ],
# Consultant en Énergie
    [
    "évaluation des besoins énergétiques",
    "audit et analyse de la consommation énergétique",
    "proposition de solutions d'efficacité énergétique",
    "mise en œuvre de mesures de réduction de la consommation",
    "gestion de projets d'optimisation énergétique",
    "utilisation de logiciels de simulation énergétique",
    "communication avec les clients et les partenaires",
    "formation et sensibilisation à l'économie d'énergie",
    "évaluation de la performance énergétique",
    "respect des réglementations et des normes environnementales"
    ],
# Analyste en Énergie
    [
    "collecte et analyse de données énergétiques",
    "modélisation et prévision de la consommation",
    "évaluation de la performance des installations",
    "identification des opportunités d'économie d'énergie",
    "rapports et recommandations pour l'efficacité énergétique",
    "utilisation de logiciels d'analyse et de simulation",
    "communication avec les clients et les décideurs",
    "suivi des tendances et des innovations énergétiques",
    "formation continue sur les méthodes et les technologies",
    "collaboration avec des équipes techniques et commerciales"
    ],
# Gestionnaire de Projet Énergie
    [
    "planification et organisation de projets énergétiques",
    "gestion des ressources et des budgets",
    "suivi des échéanciers et des objectifs",
    "coordination des équipes et des intervenants",
    "communication avec les clients et les parties prenantes",
    "gestion des risques et des imprévus",
    "rapports d'avancement et documentation des projets",
    "respect des réglementations et des normes",
    "évaluation de la performance et des résultats",
    "formation continue en gestion de projet et en énergie"
    ],
# Technicien HVAC
    [
    "installation et maintenance de systèmes de chauffage",
    "ventilation et climatisation",
    "diagnostic et dépannage des systèmes HVAC",
    "utilisation d'outils de mesure et de test",
    "lecture de plans et de schémas techniques",
    "gestion des pièces de rechange et des stocks",
    "respect des normes de sécurité et de qualité",
    "formation des utilisateurs finaux",
    "collaboration avec des équipes techniques",
    "communication avec les clients et les fournisseurs"
    ],
# Comptable
    [
    "tenue de livres et comptabilité générale",
    "analyse des états financiers",
    "déclarations fiscales et rapports réglementaires",
    "gestion des budgets et des prévisions",
    "audit comptable et contrôle interne",
    "gestion de trésorerie et recouvrement",
    "utilisation de logiciels de comptabilité",
    "communication avec les clients et les auditeurs",
    "formation continue sur les normes comptables",
    "respect de la confidentialité et de l'éthique professionnelle"
    ],
# Analyste Financier
    [
    "analyse des marchés financiers",
    "évaluation des investissements et des risques",
    "prévisions financières et modélisation",
    "rédaction de rapports d'analyse financière",
    "gestion de portefeuille et allocation d'actifs",
    "utilisation de logiciels d'analyse financière",
    "communication avec les investisseurs et les clients",
    "formation continue sur les marchés et les instruments financiers",
    "collaboration avec des équipes commerciales et de gestion",
    "respect des réglementations et des normes professionnelles"
    ],
# Banquier d'Investissement
    [
    "structuration et négociation de transactions financières",
    "évaluation des entreprises et des projets",
    "montage financier et due diligence",
    "analyse des risques et des opportunités",
    "gestion de portefeuille et de risque",
    "communication avec les clients et les partenaires",
    "utilisation de modèles financiers et d'outils d'analyse",
    "formation continue sur les marchés financiers et les produits dérivés",
    "collaboration avec des équipes juridiques et commerciales",
    "respect des réglementations et des normes éthiques"
    ],
# Contrôleur de Gestion
    [
    "analyse des performances et des écarts",
    "prévisions budgétaires et planification stratégique",
    "reporting financier et tableau de bord",
    "optimisation des coûts et des investissements",
    "gestion des budgets et des prévisions",
    "évaluation des processus et des projets",
    "communication avec les départements opérationnels",
    "utilisation de logiciels de gestion et de business intelligence",
    "formation continue sur les techniques de contrôle de gestion",
    "collaboration avec des équipes financières et opérationnelles"
    ],
# Auditeur
    [
    "audit financier et opérationnel",
    "évaluation des risques et des contrôles internes",
    "vérification de la conformité aux normes et réglementations",
    "analyse des processus et des données",
    "révision des états financiers et des rapports",
    "communication des résultats et recommandations",
    "utilisation de logiciels d'audit et de gestion des risques",
    "formation continue sur les normes et les pratiques d'audit",
    "collaboration avec des équipes d'audit interne et externe",
    "respect de l'éthique et de la confidentialité professionnelle"
    ],
# Gestionnaire de Fonds
    [
    "gestion de portefeuille et allocation d'actifs",
    "analyse des marchés financiers et des opportunités d'investissement",
    "sélection et suivi des placements",
    "relation clientèle et gestion de patrimoine",
    "communication des stratégies d'investissement",
    "utilisation de logiciels de gestion de fonds",
    "formation continue sur les tendances et les innovations en finance",
    "collaboration avec des équipes de recherche et d'analyse",
    "respect des réglementations et des normes éthiques",
    "performance et résultats financiers"
    ],
# Conseiller Financier
    [
    "analyse de la situation financière des clients",
    "planification patrimoniale et fiscale",
    "recommandations d'investissement et de gestion de patrimoine",
    "élaboration de stratégies financières personnalisées",
    "suivi et ajustement des plans financiers",
    "communication des produits financiers et des risques",
    "relation clientèle et fidélisation",
    "utilisation d'outils de simulation financière",
    "formation continue sur les produits et les marchés financiers",
    "collaboration avec des équipes de conseillers et de vente"
    ],
# Courtier
    [
    "recherche et analyse de produits financiers",
    "négociation et exécution d'ordres d'achat et de vente",
    "conseils sur les stratégies d'investissement",
    "gestion de portefeuille et de risque",
    "relation clientèle et prospection commerciale",
    "suivi des marchés financiers et des tendances",
    "utilisation de plateformes de trading et d'analyse",
    "formation continue sur les instruments financiers",
    "respect des réglementations et des normes professionnelles",
    "performance et résultats commerciaux"
    ],
# Analyste de Crédit
    [
    "évaluation de la solvabilité des emprunteurs",
    "analyse des états financiers et des données économiques",
    "notation de crédit et décision d'octroi",
    "gestion des risques et des garanties",
    "communication avec les clients et les partenaires bancaires",
    "utilisation de modèles de scoring et d'analyse",
    "formation continue sur les méthodes d'analyse de crédit",
    "collaboration avec des équipes commerciales et juridiques",
    "respect des politiques de crédit et des normes réglementaires",
    "performance et qualité des décisions de crédit"
    ],
# Gestionnaire de Risques
    [
    "identification et évaluation des risques financiers",
    "élaboration de politiques et de stratégies de gestion des risques",
    "mise en place de dispositifs de contrôle et de suivi",
    "analyse des scénarios et des impacts",
    "communication des recommandations aux décideurs",
    "utilisation d'outils d'analyse et de modélisation des risques",
    "formation continue sur les pratiques de gestion des risques",
    "collaboration avec des équipes de gestion et de conformité",
    "respect des réglementations et des normes internes",
    "performance et efficacité des dispositifs de gestion des risques"
    ],
# IA (Intelligence Artificielle)
    [
    "apprentissage automatique (Machine Learning)",
    "réseaux neuronaux et deep learning",
    "traitement du langage naturel (NLP)",
    "vision par ordinateur",
    "reconnaissance de formes et de motifs",
    "algorithmes génétiques et optimisation",
    "systèmes experts et logique floue",
    "apprentissage non supervisé et clustering",
    "utilisation de bibliothèques d'IA (TensorFlow, PyTorch)",
    "développement d'applications et de solutions IA"
    ],
# Blockchain
    [
    "technologie de registre distribué (DLT)",
    "cryptomonnaies et tokens",
    "contrats intelligents (smart contracts)",
    "minage et validation de transactions",
    "consensus et preuve de travail/autorité/participation",
    "interopérabilité entre blockchains",
    "applications décentralisées (dApps)",
    "sécurité et confidentialité des données",
    "utilisation de frameworks blockchain (Ethereum, Hyperledger)",
    "développement de solutions blockchain"
    ],
# Robotique
    [
    "conception et modélisation de robots",
    "programmation de robots et de systèmes embarqués",
    "capteurs et actionneurs",
    "navigation autonome et SLAM (Simultaneous Localization And Mapping)",
    "interaction homme-machine",
    "intelligence artificielle embarquée",
    "vision par ordinateur et reconnaissance d'objets",
    "systèmes de contrôle et de planification de mouvement",
    "applications industrielles et domestiques de la robotique",
    "collaboration avec des équipes d'ingénieurs et de chercheurs"
    ],
# Biotechnologie
    [
    "biologie moléculaire et cellulaire",
    "génétique et génomique",
    "culture de cellules et fermentation",
    "ingénierie génétique et manipulation de l'ADN",
    "bioprocédés et biofabrication",
    "analyse bioinformatique et modélisation",
    "diagnostic médical et biotechnologies médicales",
    "développement de médicaments et thérapies géniques",
    "biomatériaux et bio-impression 3D",
    "respect des normes de sécurité et des bonnes pratiques"
    ],
    [
        "développement logiciel",
        "programmation",
        "conception d'architecture",
        "gestion de projet",
        "test et débogage",
        "maîtrise des langages de programmation (Java, Python, C++)",
        "connaissance des frameworks (Spring, Django, .NET)",
        "utilisation des outils de versioning (Git, SVN)",
        "résolution de problèmes complexes",
        "collaboration en équipe"
    ],
    # Développeur Web
    [
        "développement web",
        "conception d'interfaces utilisateur",
        "langages de programmation (HTML, CSS, JavaScript)",
        "frameworks web (React, Angular, Vue.js)",
        "bases de données (SQL, MongoDB)",
        "maîtrise des outils de développement (Visual Studio Code, Webpack)",
        "gestion de projets Agile",
        "sensibilité au design et à l'expérience utilisateur",
        "résolution de problèmes techniques",
        "veille technologique"
    ],
    # Scientifique de Données
    [
        "analyse de données",
        "exploration et nettoyage de données",
        "modélisation statistique et machine learning",
        "programmation en langage R et Python",
        "utilisation de bibliothèques (NumPy, pandas, scikit-learn)",
        "visualisation de données (Matplotlib, Seaborn, Plotly)",
        "compréhension des algorithmes d'apprentissage automatique",
        "traitement de données massives (Big Data)",
        "résolution de problèmes complexes",
        "communication des résultats"
    ],
    # Cybersécurité
    [
        "sécurité informatique",
        "détection et prévention des intrusions",
        "analyse de vulnérabilités",
        "gestion des risques",
        "cryptographie",
        "sécurité des réseaux",
        "tests d'intrusion",
        "conformité réglementaire (RGPD, ISO 27001)",
        "sensibilisation à la sécurité",
        "gestion des incidents de sécurité"
    ],
    # Administrateur Réseau
    [
        "administration système",
        "configuration et maintenance des réseaux",
        "sécurité des systèmes d'information",
        "supervision et surveillance des infrastructures",
        "résolution d'incidents réseau",
        "virtualisation (VMware, Hyper-V)",
        "gestion des sauvegardes et de la reprise d'activité",
        "gestion des utilisateurs et des droits d'accès",
        "veille technologique",
        "collaboration avec les équipes techniques"
    ],
    # Développeur Mobile
    [
        "développement d'applications mobiles",
        "langages de programmation (Java, Kotlin, Swift)",
        "conception d'interfaces utilisateur mobiles",
        "utilisation des SDK (Android, iOS)",
        "déploiement sur les stores d'applications",
        "test et débogage sur différentes plateformes",
        "optimisation des performances",
        "sécurité des applications mobiles",
        "collaboration avec les équipes de design et de backend",
        "adaptabilité aux évolutions des OS mobiles"
    ],
    # Architecte Système
    [
        "architecture informatique",
        "conception de solutions techniques",
        "intégration de systèmes",
        "scalabilité et haute disponibilité",
        "sécurité des systèmes",
        "virtualisation et cloud computing",
        "gestion des bases de données",
        "optimisation des performances",
        "veille technologique",
        "communication et présentation de solutions"
    ],
    # UI/UX Designer
    [
        "design d'interfaces utilisateur (UI)",
        "expérience utilisateur (UX)",
        "wireframing et prototypage",
        "conception graphique",
        "utilisabilité et accessibilité",
        "ergonomie des interfaces",
        "tests utilisateurs",
        "outils de design (Sketch, Adobe XD, Figma)",
        "collaboration avec les développeurs et les chefs de projet",
        "créativité et sensibilité artistique"
    ],
    # Analyste Système
    [
        "analyse des besoins",
        "étude de faisabilité",
        "spécifications fonctionnelles et techniques",
        "modélisation des processus métier",
        "gestion de projet",
        "tests et recettes",
        "formation des utilisateurs",
        "support technique",
        "résolution d'incidents",
        "documentation des systèmes"
    ],
    # Ingénieur Cloud
    [
        "cloud computing",
        "architecture cloud",
        "migration vers le cloud",
        "services cloud (AWS, Azure, Google Cloud)",
        "automatisation et orchestration",
        "sécurité cloud",
        "gestion des coûts",
        "performances et scalabilité",
        "administration des plateformes cloud",
        "collaboration en équipe"
    ]
]


tokenized_competences = []

# Boucle à travers la liste des compétences par métier
for metier_competences in listeGPT_competences_nouveau:
    # Liste pour stocker les compétences tokenisées pour ce métier
    tokenized_metier_competences = []
    # Boucle à travers chaque compétence pour le métier
    for competence1 in metier_competences:
        # Tokeniser la compétence et ajouter le résultat à la liste
        tokens = teckeniser(competence1)
        tokenized_metier_competences.append(tokens)
    # Ajouter la liste des compétences tokenisées pour ce métier à la liste principale
    tokenized_competences.append(tokenized_metier_competences)

# Afficher les résultats
tokenized_competences

[[{'css', 'développ', 'html', 'javascript', 'langag', 'php', 'python', 'web'},
  {'angular',
   'asp.net',
   'django',
   'développ',
   'express',
   'flask',
   'framework',
   'laravel',
   'rail',
   'react',
   'vue.j',
   'web'},
  {'firebas', 'mongodb', 'mysql', 'postgresql', 'sql'},
  {'concept', "d'interfac"},
  set(),
  {'optimis', 'perform', 'web'},
  {'web'},
  {'test', 'unitair'},
  {'gestion', 'git', 'version'},
  {'techniqu'},
  {'développ', 'web'},
  {'design', 'respons'},
  {'seo'}],
 [{'c++', 'javascript', 'kotlin', 'langag', 'programm', 'python', 'swift'},
  {'.net',
   'angular',
   'django',
   'framework',
   'logiciel',
   'rail',
   'react',
   'spring',
   'vue.j'},
  {'desktop', 'développ', 'mobil'},
  {'architectur'},
  {'test', 'unitair'},
  {'concept', 'objet'},
  {'gestion', 'nosql'},
  {'optimis', 'perform'},
  {'bug', 'débogag'},
  {'gestion', 'logiciel', 'projet'},
  {'architectur', 'microservic'},
  {'continu'},
  {'continu', 'livraison'}],
 [{'langag

In [28]:
for sous_liste in tokenized_competences:
    for index, element in enumerate(sous_liste):
        if isinstance(element, set):
            sous_liste[index] = list(element)

tokenized_competences = [element for sous_liste in tokenized_competences for element in sous_liste]


In [29]:
tokenized_competences = [element for sous_liste in tokenized_competences for element in sous_liste]
tokenized_competences

['python',
 'javascript',
 'web',
 'php',
 'css',
 'développ',
 'html',
 'langag',
 'angular',
 'rail',
 'react',
 'laravel',
 'asp.net',
 'framework',
 'flask',
 'développ',
 'django',
 'web',
 'express',
 'vue.j',
 'firebas',
 'sql',
 'postgresql',
 'mongodb',
 'mysql',
 "d'interfac",
 'concept',
 'web',
 'perform',
 'optimis',
 'web',
 'test',
 'unitair',
 'gestion',
 'git',
 'version',
 'techniqu',
 'web',
 'développ',
 'respons',
 'design',
 'seo',
 'python',
 'javascript',
 'swift',
 'programm',
 'c++',
 'kotlin',
 'langag',
 'spring',
 'rail',
 'angular',
 'react',
 '.net',
 'logiciel',
 'framework',
 'django',
 'vue.j',
 'desktop',
 'développ',
 'mobil',
 'architectur',
 'test',
 'unitair',
 'objet',
 'concept',
 'gestion',
 'nosql',
 'perform',
 'optimis',
 'bug',
 'débogag',
 'gestion',
 'projet',
 'logiciel',
 'microservic',
 'architectur',
 'continu',
 'livraison',
 'continu',
 'python',
 'sql',
 'r',
 'programm',
 'matlab',
 'langag',
 'manipul',
 'outil',
 'pytorch',
 'ma

In [30]:
df['competence_processed'] = [[mot for mot in competence if mot in tokenized_competences] for competence in df['competence_processed']]


In [31]:
#df['competence_processed'].to_csv('competence_processed.csv', index=False)

In [32]:
df_nonempty_lists = df[df['competence_processed'].apply(lambda x: len(x) > 0)]
print(df_nonempty_lists)

            id                                           intitule  \
24283  163PSJJ                       Assistant(e) juridique (H/F)   
27675  170LBYV  FORUM RETAIL DE LUXE: CONSEILLER DE VENTE BILI...   
45423  169LXYY        Assistant / Assistante de service juridique   
40103  169LJDX                      Customer Quality Window (H/F)   
50166  169LPXZ        VENDEUR RAYON JARDIN - MAGASIN WELDOM (H/F)   
...        ...                                                ...   
42131  169HJJQ        OPERATEUR MANUTENTIONNAIRE - TAF 2024 (H/F)   
40294  167QHSL                           Opérateur chargement H/F   
28855  170PZHR                           Opérateur chargement H/F   
31211  170PTHQ                    Chef d'équipe génie civil (H/F)   
41326  168GGCS                       Opérateurs sur machine (H/F)   

                                             description  \
24283  Un cabinet d'avocats situé à MULHOUSE recrute ...   
27675  ORGANISATION FORUM RETAIL DE LUXE ORGANISE L

In [33]:
# Calculate the number of rows representing the top 10%
#df_nonempty_lists.to_csv('competence_processed_empty.csv', index=False)

In [34]:
# Calculate value counts for 'romeLibelle' in df_empty_lists
value_counts = df_nonempty_lists['romeCode'].value_counts()

# Sort the value counts in descending order
sorted_counts = value_counts.sort_values(ascending=False)

# Calculate the number of rows representing the top 10%
top_10_percent = int(0.1 * len(sorted_counts))

# Extract the top 10% biggest counts
top_10_percent_counts = sorted_counts[:top_10_percent]

# Print the top 10% biggest counts
print(top_10_percent_counts)

romeCode
I1304    696
M1203    680
K1304    655
K1302    609
I1604    559
G1601    534
G1803    524
K2204    508
N1103    491
J1501    469
D1402    461
K2503    442
G1602    363
N4101    327
I1203    324
M1607    318
H2903    309
G1203    306
D1106    293
F1703    291
C1504    290
I1307    286
D1407    285
D1403    281
F1602    274
K1207    271
D1401    263
N4105    261
J1506    255
D1507    255
H1502    246
H1206    242
M1805    241
D1202    238
F1603    228
D1102    222
J1402    219
K1104    217
D1409    210
M1601    208
F1106    208
D1214    205
K1303    202
N4103    194
H1404    187
K1201    187
D1212    180
K2107    175
G1801    174
M1401    171
H3302    171
K2111    168
Name: count, dtype: int64


In [35]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import string

def tokeniser(texte):
    words = word_tokenize(texte)
    stemmer_en = SnowballStemmer("english")
    stemmer_fr = SnowballStemmer("french")
    stopwords_fr_en = stopwords.words("english") + stopwords.words("french")
    stemmed_anglais = {stemmer_en.stem(word.lower()) for word in words if word.lower() not in stopwords_fr_en and word not in string.punctuation}
    stemmed_francais = {stemmer_fr.stem(word.lower()) for word in words if word.lower() not in stopwords_fr_en and word not in string.punctuation}
    return stemmed_anglais.union(stemmed_francais)

def appliquer_tokenisation_et_filtrage(texte, competences_tokenized_flat):
    tokens = tokeniser(texte)
    filtered_tokens = [token for token in tokens if token in competences_tokenized_flat]
    return filtered_tokens

def preparer_liste_competences(liste_competences):
    # Aplatir la liste de listes des compétences et tokeniser chaque compétence
    return {item for sublist in liste_competences for competence in sublist for item in tokeniser(competence)}

# Fonction pour traiter une entrée DataFrame
def filtrer_competences_df(df, liste_competences):
    competences_tokenized_flat = preparer_liste_competences(liste_competences)
    df['competence_processed'] = df['description'].apply(lambda x: appliquer_tokenisation_et_filtrage(x, competences_tokenized_flat))
    return df

# Fonction pour traiter un texte spécifique
def filtrer_competences_texte(texte, liste_competences):
    competences_tokenized_flat = preparer_liste_competences(liste_competences)
    return appliquer_tokenisation_et_filtrage(texte, competences_tokenized_flat)



# Exemple d'utilisation
list_competences = listeGPT_competences_nouveau
concatenated_df = pd.read_csv('../data/concatenated_offers.csv')
# Sample data
df = concatenated_df
df_processed = filtrer_competences_df(df, list_competences)


print(df_processed)


C:\Users\Moham\AppData\Local\Temp\ipykernel_13240\1243889185.py:40: DtypeWarning: Columns (39,61) have mixed types. Specify dtype option on import or set low_memory=False.
  concatenated_df = pd.read_csv('../data/concatenated_offers.csv')


            id                                           intitule  \
0      162RQFM  Technicien / Technicienne en mesures physiques...   
1      162RQCS  Agent / Agent (e) d'entretien des bâtiments et...   
2      162RQBK  Aide-soignant / Aide-soignante (H/F) ou AES de...   
3      162RPNJ                         Ambulancier Diplômé  (H/F)   
4      162RPHL             Dessinateur / Dessinatrice du bâtiment   
...        ...                                                ...   
53313  169MFPB                  Auxiliaire de Vie F/H Ville (H/F)   
53314  169MFNC            Auxiliaire de Vie F/H La Rochelle (H/F)   
53315  169MFMF                Auxiliaire de Vie F/H Périgny (H/F)   
53316  169MFFN                              AIDE A DOMICILE (H/F)   
53317  168FQDJ           Vendeur / Vendeuse en vins et spiritueux   

                                             description  \
0      Dans le cadre du développement de notre activi...   
1      Recrute en urgence pour une prise de poste i

In [36]:
# Load the Word2Vec model
from gensim.models.fasttext import FastText as FT_gensim
# Assuming the path to the downloaded 'cc.fr.300.bin' file is 'path/to/cc.fr.300.bin'

word2vec_model = FT_gensim.load_fasttext_format('C:\\Users\\Moham\\Documents\\data_pole_emploi\\Data_Pole_Emploi\\cc.fr.300.bin.gz')


C:\Users\Moham\AppData\Local\Temp\ipykernel_13240\1029641351.py:5: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  word2vec_model = FT_gensim.load_fasttext_format('C:\\Users\\Moham\\Documents\\data_pole_emploi\\Data_Pole_Emploi\\cc.fr.300.bin.gz')


In [42]:
def debug_and_validate_vectors(token_lists, model):
    num_features = model.vector_size  # The expected number of features, typically 300 for Word2Vec
    print(f"Expected feature vector size: {num_features}")

    for tokens in token_lists:
        valid_tokens = [token for token in tokens if token in model.wv.key_to_index]
        print(f"Number of valid tokens: {len(valid_tokens)} / {len(tokens)}")  # How many tokens were found in the model vocabulary

        if valid_tokens:
            model_vocabs = np.array([model.wv[token] for token in valid_tokens])
            if model_vocabs.size:
                feature_vec = np.mean(model_vocabs, axis=0)
                print(f"Generated vector size: {feature_vec.size}")  # This should match `num_features`
                if feature_vec.size != num_features:
                    print("Error: Mismatch in vector size.")
                return feature_vec  # Return for inspection
            else:
                print("No valid tokens were found.")
        else:
            print("No valid tokens after filtering.")
    return np.zeros(num_features)  # Return a zero vector if no valid tokens

# Test the function with a sample token list from your dataset
sample_tokens = df_processed['competence_processed'].iloc[0]  # Assuming this is a list of tokens
debug_vector = debug_and_validate_vectors([sample_tokens], word2vec_model)
print("Debug vector:", debug_vector)


Expected feature vector size: 300
Number of valid tokens: 47 / 50
Generated vector size: 300
Debug vector: [ 4.29448159e-03 -1.95817137e-03 -2.83781942e-02 -5.43389870e-05
 -4.83283438e-02 -1.36627667e-02  2.91835498e-02  8.55537690e-03
 -2.35176869e-02 -9.89636034e-03  2.63023726e-03 -1.55790932e-02
  1.98128782e-02  3.04630920e-02 -5.26385102e-03 -8.02759267e-03
 -1.48511734e-02 -1.64614618e-03  8.56100174e-04  5.18201524e-03
 -2.38714293e-02  9.23095550e-03  1.31244492e-02  4.59947158e-04
 -3.25050059e-04  1.42450295e-02 -4.46980745e-02 -7.88229890e-03
  2.53242627e-02  9.58722644e-03  5.36171161e-03 -1.06493123e-02
  1.18159996e-02  1.47554725e-02 -8.56599119e-03  5.53494692e-03
  2.25935951e-02 -2.13652868e-02 -1.31226946e-02 -2.01122575e-02
 -4.64551710e-03 -1.14262002e-02  3.60136246e-03  2.44659302e-03
 -1.61588565e-02 -2.59044077e-02  4.97875037e-04  1.51217226e-02
 -4.42106882e-03  7.71074847e-04  2.56051291e-02  1.25424936e-02
 -1.48831969e-02  3.29288207e-02 -5.03152097e-03

In [46]:
df["competence_processed"]

0        [nettoyag, donné, sécur, expérient, test, repo...
1        [locaux, disposit, compétenc, local, prise, bo...
2        [qualité, expérient, prise, d'hygièn, assist, ...
3        [d'hygien, intervent, sécur, sécurité, patient...
4        [utilis, suivi, bonn, autonom, évolut, compéte...
                               ...                        
53313    [qualité, besoin, entre, psychologiqu, parti, ...
53314    [qualité, besoin, entre, psychologiqu, parti, ...
53315    [qualité, besoin, entre, psychologiqu, parti, ...
53316    [servic, vi, disponibil, compétenc, capac, rec...
53317    [présentat, mouvement, suivi, présent, mise, s...
Name: competence_processed, Length: 53318, dtype: object

In [50]:
# now i want to apply the word2vec to the df result as before 

# Import the Word2Vec model
from gensim.models import Word2Vec
import pandas as pd
import numpy as np


def calculate_average_word2vec_optimized(token_lists, model):
    num_features = model.vector_size  # Typically 300 for Word2Vec models
    feature_vecs = np.zeros((len(token_lists), num_features), dtype="float32")
    for idx, tokens in enumerate(token_lists):
        valid_tokens = [token for token in tokens if token in model.wv.key_to_index]
        if valid_tokens:
            model_vocabs = np.array([model.wv[token] for token in valid_tokens])
            feature_vecs[idx] = np.mean(model_vocabs, axis=0)
    return feature_vecs


def calculate_average_word2vec_optimized(token_lists, model):
    num_features = model.vector_size  # Typically 300 for Word2Vec models
    feature_vecs = np.zeros((len(token_lists), num_features), dtype="float32")  # Prepare the output matrix

    for idx, tokens in enumerate(token_lists):
        # Gather all vectors for tokens that appear in the model's vocabulary
        valid_vectors = [model.wv[token] for token in tokens if token in model.wv.key_to_index]

        # If there are valid vectors, average them, otherwise, leave the vector as zeros
        if valid_vectors:
            mean_vector = np.mean(valid_vectors, axis=0)
            feature_vecs[idx] = mean_vector

    return feature_vecs
# Apply the mean_vector function to the DataFrame
df_processed['mean_vector'] = df_processed['competence_processed'].apply(lambda x: calculate_average_word2vec_optimized(x, word2vec_model))

# apply the same to the resume
path_cvv = "C:\\Users\\Moham\\Downloads\\cv_omar.pdf"
# use the pdf function 

texte = lirePDF(path_cvv)
# Apply the tokenisation and filtering to the resume
competences_texte = filtrer_competences_texte(texte, list_competences) # Wrap in a list to make it compatible with the function

# Calculate the mean vector of the resume
mean_vector_texte = calculate_average_word2vec_optimized([competences_texte], word2vec_model)[0]  # [0] to get the single vector out

def debug_vector_sizes(vec1, vec2):
    print(f"Length of vec1: {len(vec1)}, Length of vec2: {len(vec2)}")
    if len(vec1) != len(vec2):
        print("Vector length mismatch!")
        # Optionally, extend the shorter vector with zeros
        if len(vec1) < len(vec2):
            vec1 = np.append(vec1, np.zeros(len(vec2) - len(vec1)))
        else:
            vec2 = np.append(vec2, np.zeros(len(vec1) - len(vec2)))
        print(f"New Length of vec1: {len(vec1)}, New Length of vec2: {len(vec2)}")
    return vec1, vec2

def calculate_cosine_similarity(vec1, vec2):
    vec1, vec2 = debug_vector_sizes(vec1, vec2)  # Check and print vector sizes
    if np.linalg.norm(vec1) != 0 and np.linalg.norm(vec2) != 0:
        return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    else:
        return 0
# Assuming your vectors are stored in a DataFrame column named 'mean_vector'
vector_shapes = df_processed['mean_vector'].apply(lambda vec: np.array(vec).shape).value_counts()

# Print out all unique vector shapes found in 'mean_vector'
print("Vector shapes and their frequencies in 'mean_vector':")
print(vector_shapes)

df_processed['cosine_similarity'] = df_processed['mean_vector'].apply(
    lambda x: calculate_cosine_similarity(mean_vector_texte, x)
)
# Sort the DataFrame by cosine similarity in descending order
df_processed = df_processed.sort_values(by='cosine_similarity', ascending=False)

# Print the top 50 most similar job offers
print(df_processed.head(50))

Vector shapes and their frequencies in 'mean_vector':
mean_vector
(33, 300)     1142
(21, 300)     1077
(23, 300)     1047
(11, 300)     1012
(27, 300)     1000
              ... 
(122, 300)       1
(115, 300)       1
(121, 300)       1
(123, 300)       1
(125, 300)       1
Name: count, Length: 128, dtype: int64
Length of vec1: 300, Length of vec2: 50
Vector length mismatch!
New Length of vec1: 300, New Length of vec2: 15250


ValueError: shapes (300,) and (15250,) not aligned: 300 (dim 0) != 15250 (dim 0)

In [51]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import gensim.downloader as api

# Define your function to calculate average Word2Vec
def calculate_average_word2vec_optimized(token_lists, model):
    num_features = model.vector_size  # Typically 300 for Word2Vec models
    feature_vecs = np.zeros((len(token_lists), num_features), dtype="float32")
    for idx, tokens in enumerate(token_lists):
        valid_tokens = [token for token in tokens if token in model.wv.key_to_index]
        if valid_tokens:
            model_vocabs = np.array([model.wv[token] for token in valid_tokens])
            feature_vecs[idx] = np.mean(model_vocabs, axis=0)
    return feature_vecs

# Apply the function to your DataFrame
df_processed['mean_vector'] = df_processed['competence_processed'].apply(
    lambda tokens: calculate_average_word2vec_optimized([tokens], word2vec_model)[0]
)

# Assuming 'filtrer_competences_texte', 'lirePDF', and 'list_competences' are defined elsewhere
# and 'word2vec_model' is already loaded

texte = lirePDF(path_cvv)  # Make sure lirePDF function correctly extracts text from PDF
competences_texte = filtrer_competences_texte(texte, list_competences)

# Calculate the mean vector of the resume
mean_vector_texte = calculate_average_word2vec_optimized([competences_texte], word2vec_model)[0]

# Define a function to calculate cosine similarity
def calculate_cosine_similarity(vec1, vec2):
    if np.linalg.norm(vec1) != 0 and np.linalg.norm(vec2) != 0:  # Avoid division by zero
        return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    else:
        return 0

# Calculate cosine similarity with the mean vectors
df_processed['cosine_similarity'] = df_processed['mean_vector'].apply(
    lambda x: calculate_cosine_similarity(mean_vector_texte, x)
)

# Sort the DataFrame by cosine similarity in descending order
df_processed = df_processed.sort_values(by='cosine_similarity', ascending=False)

# Print the top 50 most similar job offers
print(df_processed.head(50))


            id                                           intitule  \
31988  170TYLF                           Data Scientist H/F (H/F)   
746    3113398                                 Data analyst (H/F)   
19817  165BDKT            Data Analyst expérimenté Business (H/F)   
44133  169HYVQ                      Docteur en Data Science (H/F)   
19919  164ZYMX                                  TEAM LEADER (H/F)   
48393  0423656                                 Data analyst (H/F)   
32274  1060180  Analyste Business Intelligence et Stratégie Co...   
33487  170TFZF  Expert Scientifique R&D Intelligence Artificie...   
35125  170KMYZ    Data-manager / Biostatisticien(ne) Senior (H/F)   
1997   2382740                                 Data manager (H/F)   
53237  169MFGG  Data analyst and Product Owner Delegate(H/F) a...   
48391  0423824                Designer / Designeuse produit (H/F)   
18695  165MDFQ  Ingénieur système solutions mécatroniques véhi...   
1934   2409932  Airbus atlantic - 

In [54]:
import os

# Set the base path for your project
base_path = "C:\\Users\\Moham\\Documents\\data_pole_emploi\\streamlit_app"

# Define the directory structure
directories = [
    "app",
    "data",
    "models",
    "notebooks"
]

# Define files to be created with their respective content
files = {
    os.path.join(base_path, "app", "__init__.py"): "",
    os.path.join(base_path, "app", "main.py"): """import streamlit as st\n""",
    os.path.join(base_path, "app", "components.py"): "",
    os.path.join(base_path, "app", "data_processing.py"): "",
    os.path.join(base_path, "app", "model.py"): "",
    os.path.join(base_path, "app", "utils.py"): "",
    os.path.join(base_path, "requirements.txt"): "streamlit\npandas\nnumpy\nnltk\ngensim\nsklearn\npdfplumber",
    os.path.join(base_path, "README.md"): "# Streamlit Application\n\nThis is a Streamlit application for processing job applications.",
    os.path.join(base_path, ".gitignore"): "*.pyc\n__pycache__/\n*.bin.gz\n*.csv\n*.ipynb",
    os.path.join(base_path, "notebooks", "exploration.ipynb"): "{}"
}

# Create directories
for directory in directories:
    dir_path = os.path.join(base_path, directory)
    os.makedirs(dir_path, exist_ok=True)
    print(f"Directory created: {dir_path}")

# Create files
for file_path, content in files.items():
    with open(file_path, 'w') as file:
        file.write(content)
    print(f"File created: {file_path}")

print("Project structure has been successfully created.")


Directory created: C:\Users\Moham\Documents\data_pole_emploi\streamlit_app\app
Directory created: C:\Users\Moham\Documents\data_pole_emploi\streamlit_app\data
Directory created: C:\Users\Moham\Documents\data_pole_emploi\streamlit_app\models
Directory created: C:\Users\Moham\Documents\data_pole_emploi\streamlit_app\notebooks
File created: C:\Users\Moham\Documents\data_pole_emploi\streamlit_app\app\__init__.py
File created: C:\Users\Moham\Documents\data_pole_emploi\streamlit_app\app\main.py
File created: C:\Users\Moham\Documents\data_pole_emploi\streamlit_app\app\components.py
File created: C:\Users\Moham\Documents\data_pole_emploi\streamlit_app\app\data_processing.py
File created: C:\Users\Moham\Documents\data_pole_emploi\streamlit_app\app\model.py
File created: C:\Users\Moham\Documents\data_pole_emploi\streamlit_app\app\utils.py
File created: C:\Users\Moham\Documents\data_pole_emploi\streamlit_app\requirements.txt
File created: C:\Users\Moham\Documents\data_pole_emploi\streamlit_app\RE

In [52]:
df_processed.head(50)


,id,intitule,description,dateCreation,dateActualisation,romeCode,romeLibelle,appellationlibelle,typeContrat,typeContratLibelle,...,contact.telephone,origineOffre.partenaires,conditionExercice,agence.telephone,Secteur,Date,Unnamed: 0,competence_processed,mean_vector,cosine_similarity
31988,170TYLF,Data Scientist H/F (H/F),"Vos missions\nActeur/Actrice du changement, vo...",2024-03-08T17:15:48.000Z,2024-03-11T11:08:40.000Z,M1403,Études et prospectives socio-économiques,Data scientist,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,49.0,2024-02-10 to 2024-03-11,NaN,"[donné, présentat, panda, besoin, data, model,...","[-0.005385068, -0.004152329, -0.029290853, 0.0...",0.963172
746,3113398,Data analyst (H/F),Description du poste :\nAu sein du pôle Market...,2023-09-28T19:58:57.000Z,2023-12-28T10:12:23.000Z,M1403,Études et prospectives socio-économiques,Data analyst,CDI,Contrat à durée indéterminée,...,NaN,"[{'nom': 'METEOJOB', 'url': 'https://www.meteo...",NaN,NaN,NaN,NaN,NaN,"[utilis, donné, présentat, d'activ, statist, n...","[0.008523706, -0.0039945617, -0.03883982, 0.01...",0.962508
19817,165BDKT,Data Analyst expérimenté Business (H/F),Si tu es animé(e) par la passion de transforme...,2023-11-20T09:58:42.000Z,2024-01-02T10:15:17.000Z,M1403,Études et prospectives socio-économiques,Data analyst,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[donné, présentat, entrepris, besoin, ajust, t...","[0.00706456, -0.015842518, -0.017460942, 0.006...",0.959804
44133,169HYVQ,Docteur en Data Science (H/F),Vos missions :\n\nRattaché à notre pôle Ingéni...,2024-02-13T10:13:30.000Z,2024-02-13T10:48:36.000Z,M1403,Études et prospectives socio-économiques,Data scientist,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,58.0,2024-01-16 to 2024-02-15,6563.0,"[donné, supervis, entrepris, psychologiqu, com...","[-0.0029842753, -0.00962766, -0.027532356, -0....",0.959633
19919,164ZYMX,TEAM LEADER (H/F),"Systnaps - Editeur français de logiciels, expe...",2023-11-18T20:50:23.000Z,2024-01-09T14:09:20.000Z,M1805,Études et développement informatique,Développeur / Développeuse informatique,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[donné, automatiqu, version, innov, framework,...","[0.0025173172, -0.0045248745, -0.03370098, -0....",0.958139
48393,0423656,Data analyst (H/F),"Barrière, c'est 33 Casinos, 1 Club de Jeux, 19...",2024-01-27T14:54:58.000Z,2024-01-27T14:54:58.000Z,M1403,Études et prospectives socio-économiques,Data analyst,CDI,Contrat à durée indéterminée,...,NaN,"[{'nom': 'TALENTPLUG', 'url': 'http://app.myta...",NaN,NaN,92.0,2024-01-16 to 2024-02-15,10923.0,"[donné, besoin, traitement, compétenc, statist...","[0.006135626, -0.00654406, -0.03262709, 0.0019...",0.957741
32274,1060180,Analyste Business Intelligence et Stratégie Co...,"La Compagnie recrute, en contrat à durée indét...",2024-02-08T13:52:17.000Z,2024-02-08T13:52:17.000Z,M1403,Études et prospectives socio-économiques,Data analyst,CDI,Contrat à durée indéterminée,...,NaN,"[{'nom': 'WE_RECRUIT', 'url': 'https://careers...",NaN,NaN,51.0,2024-02-10 to 2024-03-11,NaN,"[utilis, nettoyag, donné, besoin, analys, d'an...","[0.00082962617, -0.0097201215, -0.035182286, 0...",0.957423
33487,170TFZF,Expert Scientifique R&D Intelligence Artificie...,À propos de nous : \n\nNous sommes une équipe ...,2024-03-08T11:35:49.000Z,2024-03-09T23:18:35.000Z,H1206,"Management et ingénierie études, recherche et ...","Chef de produit études, recherche et développe...",CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,62.0,2024-02-10 to 2024-03-11,NaN,"[utilis, donné, besoin, automatiqu, innov, d'i...","[-0.0021265587, -0.004316057, -0.028474, -0.00...",0.957129
35125,170KMYZ,Data-manager / Biostatisticien(ne) Senior (H/F),Complife France est une société composée de de...,2024-03-01T17:09:41.000Z,2024-03-07T15:33:21.000Z,M1802,Expertise et support en systèmes d'information,Data manager,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,74.0,2024-02-10 to 2024-

In [53]:
#we will have the TF-IDF now 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Define the vectorizer
vectorizer = TfidfVectorizer()

# Apply the vectorizer
X = vectorizer.fit_transform(df_processed['description'])

# Convert the output to a DataFrame
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out(), index=df_processed.index)

# Print the shape of the DataFrame
print(tfidf_df.shape)

# Calculate the cosine similarity
cosine_sim = cosine_similarity(X, X)

# Convert cosine_sim to a DataFrame
cosine_sim_df = pd.DataFrame(cosine_sim, index=df_processed.index, columns=df_processed.index)

# Print the shape of the DataFrame
print(cosine_sim_df.shape)

# Print the top 50 most similar job offers
print(cosine_sim_df.head(50))


MemoryError: Unable to allocate 32.3 GiB for an array with shape (53318, 81321) and data type float64